<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/AGENTIC_TUTORIAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://datasciencedojo.com/bootcamps/agentic-ai-bootcamp/?utm_term=agentic_ai_bootcamp_discount&utm_campaign=Future%20of%20Data%20and%20AI%20Conference&utm_medium=email&_hsenc=p2ANqtz-9FXiLNaw8V3i3MXgDkM_SCN4ThUgM-rGOm2SpfiLw2IbtAzhnj-jshV0z-CYx38T6dYD8tJNoI9624tbqFG-P3pvM3Dw&_hsmi=363333365&utm_content=fodai_agentic_ai_discount&utm_source=email

In [ ]:
!pip install langchain langchain-openai beautifulsoup4 chromadb tiktoken pydantic -q
!pip install langchain_openai -q
!pip install langchain_core -q
!pip install colab-env -q

!pip install langchain_community -q

In [ ]:
import os
import colab_env

# chapter 1: Building Context-Aware LLM Applications and MCP

## 1. Understanding Model I/O: Prompts, Responses, Parsers

In [ ]:
from IPython import get_ipython
from IPython.display import display
import os
import colab_env
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from pydantic import BaseModel, Field
import json

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# --- A. Prompts ---
# Example 1: Simple Flight Search Prompt
flight_search_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant for flight planning."),
    ("user", "Find me flights from {origin} to {destination} on {date}."),
])

# Example 2: More Complex Prompt with Few-shot examples
# Escaping curly braces in example JSON to prevent them from being interpreted as variables
detailed_flight_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a flight booking assistant. Extract the origin, destination, and exact departure date (YYYY-MM-DD) from the user's request. Respond with a JSON object containing the keys 'origin', 'destination', and 'departure_date'."),
    ("human", "I want to fly from New York to London next month, specifically on July 15th."),
    ("ai", '{{"origin": "New York", "destination": "London", "departure_date": "2025-07-15"}}'), # Escaped curly braces
    ("human", "Book a trip from Paris to Rome for next Tuesday, which is June 3rd."),
    ("ai", '{{"origin": "Paris", "destination": "Rome", "departure_date": "2025-06-03"}}'), # Escaped curly braces
    ("human", "{user_query}"),
])

# --- B. Responses & C. Parsers ---

# 1. String Output Parser (default for most simple chains)
string_parser_chain = flight_search_prompt | llm | StrOutputParser()

print("--- Simple String Output ---")
response_str = string_parser_chain.invoke({"origin": "Montreal", "destination": "Toronto", "date": "July 20, 2025"})
print(f"LLM Response (String): {response_str}\n")


# 2. JSON Output Parser (for structured data)
class FlightRequest(BaseModel):
    origin: str = Field(description="The city of origin for the flight.")
    destination: str = Field(description="The city of destination for the flight.")
    departure_date: str = Field(description="The exact departure date in YYYY-MM-DD format.")

# JsonOutputParser by default returns a dictionary, even if pydantic_object is provided
json_parser_chain = detailed_flight_prompt | llm | JsonOutputParser(pydantic_object=FlightRequest)

print("--- JSON Output Parsing ---")
user_query_1 = "I need a flight from Tokyo to San Francisco on August 22nd, 2025."
# The input only needs user_query as expected by the prompt definition
parsed_flight_request_1 = json_parser_chain.invoke({"user_query": user_query_1})
print(f"Parsed Flight Request 1 (Dictionary): {parsed_flight_request_1}")
# Accessing dictionary elements using keys instead of attributes
print(f"Origin: {parsed_flight_request_1['origin']}, Destination: {parsed_flight_request_1['destination']}, Date: {parsed_flight_request_1['departure_date']}\n")

user_query_2 = "Can you find me a one-way from Berlin to Madrid for October 5th?"
parsed_flight_request_2 = json_parser_chain.invoke({"user_query": user_query_2})
print(f"Parsed Flight Request 2 (Dictionary): {parsed_flight_request_2}\n")

# Accessing dictionary elements using keys instead of attributes
print(f"Origin: {parsed_flight_request_2['origin']}, Destination: {parsed_flight_request_2['destination']}, Date: {parsed_flight_request_2['departure_date']}\n")


# Example of handling potential parsing errors (conceptual, requires more robust error handling)
print("--- Handling Malformed Responses (Conceptual) ---")
malformed_prompt = ChatPromptTemplate.from_messages([
    ("system", "Generate a JSON object with 'city' and 'country' keys. Make a mistake in the JSON structure."),
    ("human", "Give me info on London."),
])
malformed_chain = malformed_prompt | llm | StrOutputParser() # Use StrOutputParser to see the raw error
raw_malformed_response = malformed_chain.invoke({"user_query": "London"})
print(f"Raw Malformed Response: {raw_malformed_response}")
try:
    # Attempt to parse with json.loads, which would raise an error for malformed JSON
    # This is a conceptual demonstration; robust error handling would involve retries or fallback
    json.loads(raw_malformed_response)
except json.JSONDecodeError as e:
    print(f"JSON Parsing Error (as expected): {e}\n")

--- Simple String Output ---
LLM Response (String): I currently don't have access to real-time flight data or booking systems to provide specific flight options. However, I can guide you on how to find flights from Montreal to Toronto for July 20, 2025:

1. **Airline Websites**: Check the websites of airlines that frequently operate between Montreal and Toronto, such as Air Canada, WestJet, and Porter Airlines. They often have the most up-to-date schedules and prices.

2. **Online Travel Agencies (OTAs)**: Use platforms like Expedia, Kayak, Google Flights, or Skyscanner. These sites allow you to compare prices across different airlines and find the best deals.

3. **Set Alerts**: If you're planning in advance, consider setting up price alerts on these platforms. This way, you'll be notified if there are any significant changes in flight prices.

4. **Flexible Dates**: If possible, check the prices for a few days before and after July 20, 2025, as sometimes flying on a different day can

## 2. Retrieval Chains using Loaders and Retrievers

In [ ]:
!pip install langchain_community -q

In [ ]:
import os
from langchain_community.document_loaders import TextLoader, WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
import bs4

# Initialize Embeddings
embeddings = OpenAIEmbeddings()

# --- A. Loaders ---
# 1. Load from a simple text file (simulated airline policy)
# Create a dummy policy file
with open("air_canada_policy.txt", "w") as f:
    f.write("""Air Canada Baggage Policy:
    Carry-on: 1 standard item (23 x 40 x 55 cm, 10 kg max) + 1 personal item (16 x 33 x 43 cm).
    Checked Baggage: First bag up to 23 kg (50 lb) and 158 cm (62 in) total dimensions is usually free on international flights, but fees apply for domestic and additional bags.
    Excess Baggage: Overweight (23-32 kg) or oversized (158-292 cm) bags incur additional fees.
    Liquids: Liquids in carry-on must be 100ml or less per container, all fit in a 1-quart bag.
    """)

loader = TextLoader("air_canada_policy.txt")
documents = loader.load()

print("--- Loaded Documents from Text File ---")
print(f"Content of loaded document: {documents[0].page_content[:150]}...\n")

# 2. Load from a web page (conceptual, needs a real URL)
# web_loader = WebBaseLoader(
#     web_paths=("https://www.example.com/flight-info",), # Replace with a relevant URL
#     bs_kwargs=dict(
#         parse_only=bs4.SoupStrainer(["h1", "p", "li"]) # Parse only relevant tags
#     ),
# )
# web_documents = web_loader.load()
# print(f"Content of loaded web document: {web_documents[0].page_content[:150]}...\n")


# --- B. Retrievers ---
# 1. Split documents into smaller chunks for better retrieval
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

# 2. Create a vector store from the splits
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# 3. Create a retriever
retriever = vectorstore.as_retriever()

# --- C. Retrieval Chain ---
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an expert on airline policies. Use the following context to answer the user's question:\n\n{context}"),
    ("user", "{question}")
])

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

print("--- RAG Chain in Action ---")
question = "What are the carry-on baggage limits for Air Canada?"
rag_response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"RAG Answer: {rag_response}\n")

question_2 = "Can I bring a large bottle of water in my carry-on?"
rag_response_2 = rag_chain.invoke(question_2)
print(f"Question: {question_2}")
print(f"RAG Answer: {rag_response_2}\n")

# Clean up dummy file and ChromaDB persistent storage if created
if os.path.exists("air_canada_policy.txt"):
    os.remove("air_canada_policy.txt")
# if os.path.exists("chroma_db_dir"): # If you configure persistent client
#     import shutil
#     shutil.rmtree("chroma_db_dir")

--- Loaded Documents from Text File ---
Content of loaded document: Air Canada Baggage Policy:
    Carry-on: 1 standard item (23 x 40 x 55 cm, 10 kg max) + 1 personal item (16 x 33 x 43 cm).
    Checked Baggage: First ...

--- RAG Chain in Action ---
Question: What are the carry-on baggage limits for Air Canada?
RAG Answer: For Air Canada, the carry-on baggage limits are as follows: you are allowed to bring 1 standard item with maximum dimensions of 23 x 40 x 55 cm and a maximum weight of 10 kg, plus 1 personal item with dimensions of 16 x 33 x 43 cm.

Question: Can I bring a large bottle of water in my carry-on?
RAG Answer: No, you cannot bring a large bottle of water in your carry-on. According to Air Canada's baggage policy, liquids in carry-on luggage must be in containers of 100ml or less, and all containers must fit in a 1-quart bag. You would need to purchase a bottle of water after passing through security.



## 3. Implementing Memory: Buffer, Summarization, Vector-Backed

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory, ConversationSummaryMemory, ConversationBufferWindowMemory, VectorStoreRetrieverMemory
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings

# Initialize LLM and Embeddings
llm = ChatOpenAI(model="gpt-4o", temperature=0)
embeddings = OpenAIEmbeddings()

print("--- Memory Implementation ---")

# --- A. Buffer Memory ---
print("\n--- 1. Conversation Buffer Memory ---")
# Stores all messages up to the context window limit
buffer_memory = ConversationBufferMemory()
buffer_conversation = ConversationChain(llm=llm, memory=buffer_memory, verbose=False)

print("User: Hello, I'm planning a flight.")
buffer_conversation.predict(input="Hello, I'm planning a flight.")
print("AI: " + buffer_conversation.memory.buffer) # Accessing memory directly for demonstration

print("User: I want to go to Paris.")
buffer_conversation.predict(input="I want to go to Paris.")
print("AI: " + buffer_conversation.memory.buffer)

print("User: From Montreal.")
buffer_conversation.predict(input="From Montreal.")
print("AI: " + buffer_conversation.memory.buffer) # Shows full history


print("\n--- 2. Conversation Buffer Window Memory (Last N interactions) ---")
window_memory = ConversationBufferWindowMemory(k=2) # Keep last 2 exchanges
window_conversation = ConversationChain(llm=llm, memory=window_memory, verbose=False)

print("User: Hi there.")
window_conversation.predict(input="Hi there.")

print("User: My name is Alice.")
window_conversation.predict(input="My name is Alice.")

print("User: I need to book a flight.")
window_conversation.predict(input="I need to book a flight.")

print("User: From NYC to London.")
window_conversation.predict(input="From NYC to London.")
# Check the memory, it should only contain the last 2 turns
print("Window Memory Buffer:")
print(window_conversation.memory.buffer) # Should show last two turns


# --- B. Summarization Memory ---
print("\n--- 3. Conversation Summary Memory ---")
summary_llm = ChatOpenAI(model="gpt-4o", temperature=0) # Can use a cheaper model for summarization
summary_memory = ConversationSummaryMemory(llm=summary_llm)
summary_conversation = ConversationChain(llm=llm, memory=summary_memory, verbose=False)

print("User: I'm looking for a flight to Tokyo.")
summary_conversation.predict(input="I'm looking for a flight to Tokyo.")

print("User: I prefer flights on ANA or JAL.")
summary_conversation.predict(input="I prefer flights on ANA or JAL.")

print("User: And it should be for late next year, perhaps December.")
summary_conversation.predict(input="And it should be for late next year, perhaps December.")

print("Current Summary Memory:")
print(summary_conversation.memory.buffer) # The summary will update as conversation progresses


# --- C. Vector-Backed Memory ---
print("\n--- 4. Vector-Backed Memory ---")
# This creates an in-memory ChromaDB for demonstration. For persistence, configure Chroma to a directory.
# In a real app, you'd use a persistent vector store.
vectorstore_for_memory = Chroma(collection_name="chat_history", embedding_function=embeddings)
# Create a dummy retriever. In a real scenario, you'd add documents to it.
retriever_for_memory = vectorstore_for_memory.as_retriever(search_kwargs={"k": 1})

# To demonstrate, let's manually add some "past conversations"
vectorstore_for_memory.add_texts(
    ["User said they prefer window seats.",
     "User asked about pet travel policies on their last trip.",
     "User's budget for international flights is usually under $1000."]
)


vector_memory = VectorStoreRetrieverMemory(retriever=retriever_for_memory)
vector_conversation = ConversationChain(llm=llm, memory=vector_memory, verbose=False)

print("User: Do you remember my seating preference?")
current_input_1 = "Do you remember my seating preference?"
response = vector_conversation.predict(input=current_input_1)
print(f"AI: {response}")
# Pass the actual input to load_memory_variables to see what was retrieved for THIS input
print(f"Relevant context retrieved by vector memory: {vector_memory.load_memory_variables({'input': current_input_1})['history']}\n")

print("User: What about the pet policy from last time?")
current_input_2 = "What about the pet policy from last time?"
response = vector_conversation.predict(input=current_input_2)
print(f"AI: {response}")
# Pass the actual input to load_memory_variables to see what was retrieved for THIS input
print(f"Relevant context retrieved by vector memory: {vector_memory.load_memory_variables({'input': current_input_2})['history']}\n")


# Clean up ChromaDB if it created local files (depends on how you initialized it)
# For in-memory Chroma, no cleanup needed. For persistent Chroma, you'd delete the directory.
# Example: If you had `Chroma(persist_directory="./chroma_data", ...)`
# import shutil
# if os.path.exists("./chroma_data"):
#     shutil.rmtree("./chroma_data")

--- Memory Implementation ---

--- 1. Conversation Buffer Memory ---
User: Hello, I'm planning a flight.


<ipython-input-9-e1c99b271c44>:17: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  buffer_memory = ConversationBufferMemory()
<ipython-input-9-e1c99b271c44>:18: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use :meth:`~RunnableWithMessageHistory: https://python.langchain.com/v0.2/api_reference/core/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html` instead.
  buffer_conversation = ConversationChain(llm=llm, memory=buffer_memory, verbose=False)


AI: Human: Hello, I'm planning a flight.
AI: Hello! That sounds exciting. Are you planning a trip for business or pleasure? If you need any tips on booking flights, choosing airlines, or packing efficiently, feel free to ask!
User: I want to go to Paris.
AI: Human: Hello, I'm planning a flight.
AI: Hello! That sounds exciting. Are you planning a trip for business or pleasure? If you need any tips on booking flights, choosing airlines, or packing efficiently, feel free to ask!
Human: I want to go to Paris.
AI: Paris is a wonderful destination! Whether you're interested in exploring its rich history, indulging in its world-renowned cuisine, or simply enjoying the romantic atmosphere, there's something for everyone. When planning your flight, consider flying into Charles de Gaulle Airport (CDG), which is the largest international airport in France and well-connected to the city center. Alternatively, Orly Airport (ORY) is another option, located slightly closer to the city.

When booking 

<ipython-input-9-e1c99b271c44>:34: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  window_memory = ConversationBufferWindowMemory(k=2) # Keep last 2 exchanges


User: My name is Alice.
User: I need to book a flight.
User: From NYC to London.
Window Memory Buffer:
Human: I need to book a flight.
AI: That sounds exciting! Booking a flight can be quite the adventure. Here are a few tips to help you get started:

1. **Choose Your Destination and Dates**: First, decide where and when you want to travel. Flexibility with dates can sometimes help you find better deals.

2. **Use Flight Comparison Websites**: Websites like Google Flights, Skyscanner, or Kayak can help you compare prices across different airlines and find the best deals.

3. **Consider Nearby Airports**: Sometimes flying into or out of a nearby airport can save you money.

4. **Check for Discounts and Deals**: Look for any available discounts, such as student or senior discounts, or special promotions.

5. **Book in Advance**: Generally, booking a few months in advance can help you secure better prices, especially for international flights.

6. **Read the Fine Print**: Make sure to che

<ipython-input-9-e1c99b271c44>:56: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  summary_memory = ConversationSummaryMemory(llm=summary_llm)


User: I prefer flights on ANA or JAL.
User: And it should be for late next year, perhaps December.
Current Summary Memory:
The human is looking for a flight to Tokyo for late next year, preferably in December, and prefers flights on ANA or JAL. The AI suggests considering major airlines like Japan Airlines, All Nippon Airways (ANA), Delta, and United, and recommends comparing prices on travel websites such as Expedia, Kayak, or Google Flights. The AI advises booking early for December, a popular travel period, and highlights ANA's modern fleet and exceptional in-flight service, and JAL's hospitality and attention to detail. It suggests checking ANA's "Inspiration of Japan" service and JAL's "Sky Suite" for unique experiences, and recommends checking their official websites for exclusive promotions and signing up for their frequent flyer programs, ANA Mileage Club or JAL Mileage Bank, for frequent travelers. The AI also advises signing up for fare alerts on travel websites to be notifie

<ipython-input-9-e1c99b271c44>:76: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore_for_memory = Chroma(collection_name="chat_history", embedding_function=embeddings)
<ipython-input-9-e1c99b271c44>:88: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  vector_memory = VectorStoreRetrieverMemory(retriever=retriever_for_memory)


User: Do you remember my seating preference?
AI: Yes, you mentioned that you prefer window seats. It's a great choice for enjoying the view and having a bit more privacy during your travels! If you're planning a trip, I can help with tips on how to secure a window seat or make your journey more comfortable.
Relevant context retrieved by vector memory: input: Do you remember my seating preference?
response: Yes, you mentioned that you prefer window seats. It's a great choice for enjoying the view and having a bit more privacy during your travels! If you're planning a trip, I can help with tips on how to secure a window seat or make your journey more comfortable.

User: What about the pet policy from last time?
AI: On your last trip, the pet policy allowed small pets to travel in the cabin with you, provided they were in an airline-approved carrier that fits under the seat in front of you. The combined weight of the pet and the carrier couldn't exceed 20 pounds. For larger pets, they had

## 4. Combining Modules into Coherent, State-Aware Workflows

In [ ]:
import os
import colab_env

In [ ]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_core.tools import tool
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional # Import Optional
from langchain.agents import create_tool_calling_agent, AgentExecutor

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# --- Define Tools (Simulated APIs) ---

class FlightSearchParams(BaseModel):
    origin: str = Field(description="The departure city or airport code.")
    destination: str = Field(description="The arrival city or airport code.")
    departure_date: str = Field(description="The desired departure date in YYYY-MM-DD format.")
    # Make return_date optional
    return_date: Optional[str] = Field(description="Optional: The desired return date in YYYY-MM-DD format, if applicable.", default=None)
    passengers: int = Field(description="Number of passengers.", default=1)

# Make the description of the search tool very clear about its purpose
@tool("flight_search_tool", args_schema=FlightSearchParams)
def flight_search_tool(origin: str, destination: str, departure_date: str, return_date: str = None, passengers: int = 1) -> List[Dict[str, Any]]:
    """
    **PRIMARY FUNCTIONALITY: FINDING FLIGHTS.** Searches for available flight options based on origin, destination, dates, and number of passengers. Returns a list of simulated flight options (like AC123, AF456, etc.) but DOES NOT book them. Use this tool ONLY for searching/finding available flights.
    """
    print(f"\n--- Calling Flight Search API ---")
    print(f"Searching flights: {origin} -> {destination}, Dep: {departure_date}, Ret: {return_date}, Paxs: {passengers}")

    # Simulate API response
    if "montreal" in origin.lower() and "paris" in destination.lower() and "2025-08-15" in departure_date:
        return [
            {"flight_id": "AC123", "airline": "Air Canada", "price": 750, "departure_time": "09:00", "arrival_time": "18:00"},
            {"flight_id": "AF456", "airline": "Air France", "price": 820, "departure_time": "10:30", "arrival_time": "19:30"},
        ]
    elif "tokyo" in origin.lower() and "london" in destination.lower() and "2025-09-01" in departure_date:
        return [
            {"flight_id": "BA789", "airline": "British Airways", "price": 1200, "departure_time": "14:00", "arrival_time": "22:00"},
        ]
    else:
        return []

class BookingConfirmationParams(BaseModel):
    flight_id: str = Field(description="The unique identifier (like AC123 or BA789) of the specific flight you want to book. THIS IS REQUIRED FOR BOOKING.")
    passengers: int = Field(description="Number of passengers for the booking.")
    user_name: str = Field(description="Name of the person making the booking.")

# Make the description of the booking tool very clear about its purpose and required parameters
@tool("book_flight_tool", args_schema=BookingConfirmationParams)
def book_flight_tool(flight_id: str, passengers: int, user_name: str) -> str:
    """
    **PRIMARY FUNCTIONALITY: BOOKING A SPECIFIC FLIGHT.** Books a specific flight using its unique flight_id, number of passengers, and user name. Use this tool ONLY when the user provides a specific flight ID and explicitly asks to book.
    """
    print(f"\n--- Calling Booking API ---")
    print(f"Booking Flight ID: {flight_id} for {passengers} passengers, Name: {user_name}")
    # Simulate success/failure
    if "AC" in flight_id:
        return f"Booking confirmed for flight {flight_id} for {passengers} passengers under {user_name}. Reference: AC-BOOK-12345."
    else:
        return f"Booking failed for flight {flight_id}. Please try again."

# --- Define the Agent and Tools ---
tools = [flight_search_tool, book_flight_tool]

# We'll use a simple agent executor for demonstration.
# For complex state-aware workflows, LangGraph or custom chains are better.

# Memory for the conversation (used by the agent)
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Prompt for the agent - slightly refine instructions
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI flight booking assistant. Your primary goal is to help users find and book flights using the provided tools. "
               "Use the 'flight_search_tool' when the user asks to *find* or *search* for flights, providing origin, destination, and date. "
               "Use the 'book_flight_tool' when the user explicitly asks to *book* a specific flight AND provides a 'flight_id' (like AC123), number of passengers, and user name. "
               "For any questions about our *current conversation history*, answer ONLY based on the chat history provided below. DO NOT use tools to answer questions about previous interactions. "
               "If you need more information to use a tool (e.g., return date, number of passengers, or a specific flight_id for booking), ask clarifying questions. Maintain conversation history. Current time is 2025-05-26."),
    ("placeholder", "{chat_history}"), # Place memory before the user input and scratchpad
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}"), # For intermediate agent steps
])

# Create the agent (rest remains the same)
agent = create_tool_calling_agent(llm, tools, agent_prompt)

# Create the agent executor (enable verbose to see tool calls)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)


print("--- Flight Booking Workflow (Agent with Memory and Tools) ---")

print("User: I need to book a flight from Montreal to Paris.")
response_1 = agent_executor.invoke({"input": "I need to book a flight from Montreal to Paris."})
print(f"AI Response: {response_1['output']}\n")

print("User: For August 15th, 2025.")
response_2 = agent_executor.invoke({"input": "For August 15th, 2025."})
print(f"AI Response: {response_2['output']}\n")

# This is the problematic step - the agent should call book_flight_tool
print("User: Book flight AC123 for 2 people, my name is John Doe.")
response_3 = agent_executor.invoke({"input": "Book flight AC123 for 2 people, my name is John Doe."})
print(f"AI Response: {response_3['output']}\n")

# This question should now be answered from memory without tool use.
print("User: What was the origin of my first flight search?")
response_4 = agent_executor.invoke({"input": "What was the origin of my first flight search?"})
print(f"AI Response: {response_4['output']}\n")

print("\n--- Another scenario ---")
print("User: Find me a flight from Tokyo to London for Sep 1st, 2025.")
response_5 = agent_executor.invoke({"input": "Find me a flight from Tokyo to London for Sep 1st, 2025."})
print(f"AI Response: {response_5['output']}\n")

--- Flight Booking Workflow (Agent with Memory and Tools) ---
User: I need to book a flight from Montreal to Paris.


> Entering new AgentExecutor chain...
To assist you with booking a flight from Montreal to Paris, I'll first need to find available flights for you. Could you please provide the following details:

1. Departure date
2. Return date (if applicable)
3. Number of passengers

Once I have this information, I can search for available flights.

> Finished chain.
AI Response: To assist you with booking a flight from Montreal to Paris, I'll first need to find available flights for you. Could you please provide the following details:

1. Departure date
2. Return date (if applicable)
3. Number of passengers

Once I have this information, I can search for available flights.

User: For August 15th, 2025.


> Entering new AgentExecutor chain...
How many passengers will be traveling? Also, do you need a return flight, or is it a one-way trip?

> Finished chain.
AI Response: How many pa

## 5. An Introduction to Model Context Protocol (MCP)

Implementing MCP directly in a simple Python script is not feasible as it's a protocol for system-level integration and data exchange. However, I can provide a conceptual overview and how you'd think about it in the context of our flight planning agent.

Conceptual Understanding and Integration points:

MCP aims to standardize how AI models can securely and reliably:

* Access External Data: For a flight planning agent, this means getting real-time flight data, airport information, airline policies, user profiles, etc., from various external systems.

* Execute Functions/Tools: Calling specific APIs (like our flight_search_tool or book_flight_tool in the example above) in a standardized way.

* Manage Context and State: MCP could facilitate sharing context (like parsed user requests or memory summaries) between different components or models if your system uses a microservice architecture.

How it relates to our code:

In our "Combining Modules" example, flight_search_tool and book_flight_tool are essentially proxies for external API calls. With MCP, these interactions would be formalized.

In [ ]:
# --- Conceptual MCP Integration Points ---

# Instead of direct Python function calls, imagine these are MCP-compliant service endpoints.

# 1. External Flight Data Service (via MCP)
# This service would expose a standardized interface for querying flight data.
# Your LLM agent, when needing flight info, would formulate an MCP request (e.g., a function call
# request with specific parameters) to this service.
def mcp_flight_data_service_call(origin, destination, date, return_date=None, passengers=1):
    """
    Conceptual: This function represents an interaction with an MCP-compliant
    flight data service. It would send a standardized request and receive a
    standardized response.
    """
    print(f"[MCP] Requesting flight data for {origin} to {destination}...")
    # In reality, this would involve HTTP requests to an MCP server,
    # parsing structured responses, and handling standardized errors.
    # For now, it just calls our simulated tool's underlying function:
    # Corrected: Call the original function wrapped by the tool decorator
    return flight_search_tool.func(origin=origin, destination=destination, departure_date=date, return_date=return_date, passengers=passengers)

# 2. User Profile Service (via MCP)
# Storing and retrieving user preferences (e.g., preferred airlines, seat preferences,
# past travel history) could be handled by an MCP-compliant user profile service.
def mcp_user_profile_service_call(user_id, query):
    """
    Conceptual: Interacts with an MCP-compliant user profile service to retrieve
    or update user information.
    """
    print(f"[MCP] Querying user profile for ID: {user_id} with query: {query}")
    # This might return JSON or a specific data structure based on the MCP spec.
    if "seating preference" in query.lower():
        return {"preference": "window seat"}
    return {"message": "No specific preference found."}

# 3. Booking Confirmation Service (via MCP)
# The final booking step would interact with an MCP-compliant booking service.
def mcp_booking_service_call(flight_id, passengers, user_name):
    """
    Conceptual: Interacts with an MCP-compliant booking service to finalize a booking.
    """
    print(f"[MCP] Sending booking request for {flight_id}...")
    # Corrected: Call the original function wrapped by the tool decorator
    return book_flight_tool.func(flight_id=flight_id, passengers=passengers, user_name=user_name)

# How your agent would use it (conceptual update to the agent_executor)
# The agent would still use the "tools" concept, but the tools' implementations
# would internally be making MCP-compliant calls rather than direct function calls.

# Example: The `flight_search_tool` would be redefined to call `mcp_flight_data_service_call`
# internally.
# @tool("flight_search_tool", args_schema=FlightSearchParams)
# def flight_search_tool_mcp_enabled(origin: str, destination: str, departure_date: str, return_date: str = None, passengers: int = 1) -> List[Dict[str, Any]]:
#     """
#     Searches for available flights using an MCP-compliant service.
#     """
#     return mcp_flight_data_service_call(origin, destination, departure_date, return_date, passengers)

print("\n--- Conceptual MCP Integration Points Demonstrated ---")
print("This section explains how our previous tools would conceptually interact with MCP.")
print("The agent framework (LangChain) would orchestrate calls to these 'MCP-enabled' tools.")
print("The actual MCP implementation involves more than just function calls; it defines")
print("standardized data formats, communication protocols, and security measures for AI-tool interaction.")
# These calls should now work as they call the underlying function
mcp_flight_data_service_call("Montreal", "Paris", "2025-08-15")
print(mcp_user_profile_service_call("user_123", "seating preference"))


--- Conceptual MCP Integration Points Demonstrated ---
This section explains how our previous tools would conceptually interact with MCP.
The agent framework (LangChain) would orchestrate calls to these 'MCP-enabled' tools.
The actual MCP implementation involves more than just function calls; it defines
standardized data formats, communication protocols, and security measures for AI-tool interaction.
[MCP] Requesting flight data for Montreal to Paris...

--- Calling Flight Search API ---
Searching flights: Montreal -> Paris, Dep: 2025-08-15, Ret: None, Paxs: 1
[MCP] Querying user profile for ID: user_123 with query: seating preference
{'preference': 'window seat'}


# chapter 2: Vector Databases

Okay, let's dive deeper into efficient vector storage and retrieval, which is absolutely critical for building performant and accurate RAG (Retrieval Augmented Generation) systems in our flight planning AI agent. This allows the LLM to access vast amounts of external knowledge quickly and relevantly.

We'll continue using langchain, chromadb, and openai for demonstration.

Efficient Vector Storage and Retrieval
Rationale for Vector Databases:

As our AI agent for flight planning needs to access airline policies, airport information, specific flight details, and potentially historical user preferences, storing all this information within the LLM's context window is impossible. Traditional databases aren't optimized for semantic similarity searches ("find documents like this query").

Vector databases (or vector stores) solve this by:

1. Semantic Search: Storing data as high-dimensional vectors (embeddings) that capture semantic meaning. Queries are also converted to vectors, and the database finds vectors that are "close" to the query vector in this space, indicating semantic similarity.
2. Scalability: Designed to handle billions of vectors and perform lightning-fast similarity searches across them.
3. Efficiency: Employing specialized indexing algorithms to speed up searches, even over massive datasets.

## 1. Rationale for Vector Databases

Prerequisites & Setup:

In [ ]:
import os
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from typing import List, Dict, Tuple
from collections import defaultdict
import numpy as np

# Ensure your OpenAI API key is set
# os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"

# Initialize LLM and Embeddings
llm = ChatOpenAI(model="gpt-4o", temperature=0)
embeddings = OpenAIEmbeddings()

# --- Helper function to create a dummy vector store for demonstrations ---
def setup_vectorstore_with_dummy_data(collection_name="flight_docs"):
    """Creates a ChromaDB with some dummy flight-related documents."""
    texts = [
        "Air Canada allows 1 carry-on bag (max 10kg) and 1 personal item. Checked bags up to 23kg are often free for international flights.",
        "Baggage policy for domestic flights with Air Canada: first checked bag costs $30, second $50.",
        "Montreal Pierre Elliott Trudeau International Airport (YUL) is a major hub in Quebec, Canada, serving many international destinations.",
        "Paris Charles de Gaulle Airport (CDG) is France's largest international airport.",
        "To travel with a pet on Air Canada, it must be in an approved carrier, and fees apply. Check specific dimensions.",
        "Refunds for flight cancellations depend on the fare type and airline policy. Basic fares are often non-refundable.",
        "Delta Airlines has a strict policy: carry-on must fit in the overhead bin. Checked bags incur fees on all domestic flights.",
        "American Airlines offers complimentary snacks on long-haul international flights.",
        "When connecting through Toronto Pearson (YYZ), ensure you have enough layover time, especially for international to domestic transfers.",
        "Flight delays due to weather are not typically compensated by airlines. Check your travel insurance.",
        "You can find your flight status on the airline's website or airport's live departure boards.",
        "Loyalty programs like Aeroplan (Air Canada) offer points for flights and upgrades.",
        "Upgrading your seat on a flight often depends on availability and your loyalty status.",
        "Economy class amenities on long international flights include meals and in-flight entertainment.",
        "Business class passengers get priority boarding, lounge access, and lie-flat seats on most wide-body aircraft."
    ]
    metadatas = [
        {"source": "AirCanadaPolicy", "type": "baggage", "airline": "Air Canada"},
        {"source": "AirCanadaPolicy", "type": "baggage", "airline": "Air Canada"},
        {"source": "AirportInfo", "type": "airport", "city": "Montreal"},
        {"source": "AirportInfo", "type": "airport", "city": "Paris"},
        {"source": "AirCanadaPolicy", "type": "pets", "airline": "Air Canada"},
        {"source": "GeneralPolicy", "type": "cancellation"},
        {"source": "DeltaPolicy", "type": "baggage", "airline": "Delta Airlines"},
        {"source": "AmericanAirlines", "type": "amenities", "airline": "American Airlines"},
        {"source": "AirportInfo", "type": "airport", "city": "Toronto"},
        {"source": "GeneralPolicy", "type": "delays"},
        {"source": "FlightStatus", "type": "info"},
        {"source": "LoyaltyProgram", "type": "points"},
        {"source": "FlightExperience", "type": "upgrade"},
        {"source": "FlightExperience", "type": "amenities"},
        {"source": "FlightExperience", "type": "amenities"},
    ]

    # Split documents (even though they are small, good practice)
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
    docs = text_splitter.create_documents(texts, metadatas=metadatas)

    # Create ChromaDB from documents
    # Using a client with persist_directory for potential re-use or inspection
    vectorstore = Chroma.from_documents(
        documents=docs,
        embedding=embeddings,
        collection_name=collection_name,
        persist_directory=f"./{collection_name}_db" # This will create a local directory
    )
    return vectorstore

# Setup our vector store
flight_vectorstore = setup_vectorstore_with_dummy_data()
flight_retriever = flight_vectorstore.as_retriever()
print(f"ChromaDB initialized with {len(flight_vectorstore.get()['ids'])} documents.\n")

ChromaDB initialized with 15 documents.



## 2. Vector Search, Text Search, Hybrid Search

* Vector Search (Semantic Search): Finds documents based on semantic similarity using embeddings.
* Text Search (Keyword Search): Finds documents based on exact keyword matches. Often implemented using traditional inverted indexes (like in Elasticsearch, Solr).
* Hybrid Search: Combines both vector and text search to leverage the strengths of each, often leading to more relevant results.

In [ ]:
print("--- Text Search (Simulated/Conceptual Keyword Search) ---")
# Simulating a keyword search by filtering on metadata or checking content manually
query_keyword = "Air Canada baggage policy"
# A real text search would return documents with these keywords directly.
# Here, we'll demonstrate a semantic search that happens to hit relevant keywords,
# or filter by metadata if we indexed it that way.

# FIX: Structure the filter using the $and operator for multiple conditions
keyword_simulated_results = flight_vectorstore.similarity_search(
    query_keyword,
    k=5,
    filter={"$and": [{"airline": "Air Canada"}, {"type": "baggage"}]} # Use $and operator
)

print(f"Query: '{query_keyword}' (Simulated Keyword Match via Metadata/Semantic)")
for i, doc in enumerate(keyword_simulated_results):
    print(f"Result {i+1} (Keyword Sim.): {doc.page_content[:100]}... (Source: {doc.metadata.get('source')})")
print("\n")


# --- Hybrid Search using Reciprocal Rank Fusion (RRF) ---
# RRF is a method to combine ranked lists from different search methods (e.g., vector search and keyword search)
# without requiring score normalization. It works by assigning a score based on the reciprocal of the rank.

def reciprocal_rank_fusion(
    ranked_lists: List[List[Tuple[str, float]]], k=60
) -> List[Tuple[str, float]]:
    """
    Applies Reciprocal Rank Fusion to a list of ranked lists of (document_id, score) tuples.
    Ranks documents from multiple sources. Higher scores mean higher relevance.
    """
    fused_scores = defaultdict(float)
    document_counts = defaultdict(int)

    for ranked_list in ranked_lists:
        for rank, (doc_id, score) in enumerate(ranked_list):
            fused_scores[doc_id] += 1 / (k + rank)
            document_counts[doc_id] += 1

    # Sort documents by their fused scores in descending order
    # It's important to return actual documents, not just IDs.
    # For this demo, we'll assume the 'doc_id' is the content itself for simplicity,
    # or you'd map back to full Document objects.
    fused_ranked_docs = sorted(fused_scores.items(), key=lambda item: item[1], reverse=True)
    return fused_ranked_docs

print("--- Hybrid Search (Conceptual RRF) ---")

# Step 1: Perform vector search
query = "What about bringing my cat on the flight?"
vector_results_with_scores = flight_vectorstore.similarity_search_with_score(query, k=5)
# Convert to (content, score) for RRF
vector_rank_list = [(doc.page_content, score) for doc, score in vector_results_with_scores]
print("Vector Search Ranks:")
for i, (content, score) in enumerate(vector_rank_list):
    print(f"  {i+1}. {content[:50]}... (Score: {score:.4f})")

# Step 2: Simulate keyword search results (we'll manually pick some relevant ones based on keywords)
# In a real system, this would come from a keyword index.
keyword_search_simulated_docs = [
    "To travel with a pet on Air Canada, it must be in an approved carrier, and fees apply. Check specific dimensions.",
    "Air Canada allows 1 carry-on bag (max 10kg) and 1 personal item. Checked bags up to 23kg are often free for international flights.", # Less relevant, but contains "Air Canada"
    "Delta Airlines has a strict policy: carry-on must fit in the overhead bin. Checked bags incur fees on all domestic flights."
]
# Assign arbitrary scores/ranks for simulation (higher score = higher assumed relevance by keyword)
keyword_rank_list = [
    (keyword_search_simulated_docs[0], 0.95),
    (keyword_search_simulated_docs[1], 0.8),
    (keyword_search_simulated_docs[2], 0.7)
]
print("\nKeyword Search Ranks (Simulated):")
for i, (content, score) in enumerate(keyword_rank_list):
    print(f"  {i+1}. {content[:50]}... (Score: {score:.4f})")

# Step 3: Apply RRF
combined_ranks = reciprocal_rank_fusion([vector_rank_list, keyword_rank_list])

print(f"\nHybrid Search Results (RRF for query: '{query}'):")
# Need to map back to full Document objects if you want metadata
# For this demo, we're just printing the content and fused score
for i, (content, fused_score) in enumerate(combined_ranks[:5]): # Show top 5 fused results
    print(f"  {i+1}. {content[:100]}... (Fused Score: {fused_score:.4f})")
print("\n")

--- Text Search (Simulated/Conceptual Keyword Search) ---
Query: 'Air Canada baggage policy' (Simulated Keyword Match via Metadata/Semantic)
Result 1 (Keyword Sim.): Air Canada allows 1 carry-on bag (max 10kg) and 1 personal item. Checked bags up to 23kg are often f... (Source: AirCanadaPolicy)
Result 2 (Keyword Sim.): Baggage policy for domestic flights with Air Canada: first checked bag costs $30, second $50.... (Source: AirCanadaPolicy)


--- Hybrid Search (Conceptual RRF) ---
Vector Search Ranks:
  1. To travel with a pet on Air Canada, it must be in ... (Score: 0.3503)
  2. American Airlines offers complimentary snacks on l... (Score: 0.4198)
  3. Delta Airlines has a strict policy: carry-on must ... (Score: 0.4224)
  4. Economy class amenities on long international flig... (Score: 0.4333)
  5. Air Canada allows 1 carry-on bag (max 10kg) and 1 ... (Score: 0.4385)

Keyword Search Ranks (Simulated):
  1. To travel with a pet on Air Canada, it must be in ... (Score: 0.9500)
  2. Air 

## 3. Indexing Techniques & Retrieval Methods

* Product Quantization (PQ): A compression technique that reduces the memory footprint of vectors and speeds up search by breaking vectors into subvectors and quantizing each subvector. Search then operates on these compressed representations.

* Locality Sensitive Hashing (LSH): A technique that hashes similar items to the same "buckets" with high probability, reducing the number of candidate vectors to compare.


* Hierarchical Navigable Small World(HNSW): An approximate nearest neighbor (ANN) algorithm that builds a multi-layer graph structure, allowing for very fast and accurate nearest neighbor searches. It's widely used in modern vector databases (Chroma, Faiss, Weaviate, Pinecone, etc.).



Retrieval:

* Cosine Similarity: Measures the cosine of the angle between two non-zero vectors. A value close to 1 means high similarity, 0 means orthogonality, and -1 means strong dissimilarity. It's a common metric for semantic similarity.

* Nearest Neighbor Search: The core operation of vector databases, finding the vectors (documents) that are "closest" to the query vector in the high-dimensional space, based on a distance metric (like cosine similarity or Euclidean distance).


Code Note: These techniques (PQ, LSH, HNSW, Cosine Similarity, Nearest Neighbor Search) are typically implemented internally by the vector database itself (e.g., ChromaDB, Faiss, Pinecone, Weaviate). As users of LangChain, we interact with the retriever abstraction, and the underlying vector database handles the efficient indexing and search using these algorithms. You usually don't write explicit code for HNSW graph traversal or PQ quantization in your application logic; you configure the vector database (e.g., during initialization or indexing).

For instance, when you create Chroma.from_documents or vectorstore.as_retriever(), Chroma is internally using efficient algorithms like HNSW to perform the nearest neighbor search.

In [ ]:
print("--- Indexing Techniques & Retrieval Methods (Conceptual) ---")
print("Vector databases like Chroma, Faiss, Pinecone, etc., internally use advanced indexing algorithms")
print("such as HNSW (Hierarchical Navigable Small World) for fast Approximate Nearest Neighbor (ANN) search.")
print("Compression techniques like Product Quantization (PQ) and Locality Sensitive Hashing (LSH)")
print("are also handled internally by these databases to optimize storage and speed.")
print("Retrieval primarily relies on similarity metrics like Cosine Similarity or Euclidean Distance.")
print("When we call `retriever.invoke(query)` or `vectorstore.similarity_search()`, these algorithms are at work.\n")

# Example of configuring a specific search type if the retriever supports it (e.g., MMR)
# MMR (Maximal Marginal Relevance) is a diversification technique often supported by retrievers
# It tries to select documents that are relevant to the query AND diverse from each other.
print("--- Maximal Marginal Relevance (MMR) Retrieval ---")
mmr_retriever = flight_vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 5, "fetch_k": 10, "lambda_mult": 0.5})
# k: number of results to return
# fetch_k: number of results to fetch before re-ranking for diversity
# lambda_mult: how much to prioritize diversity over relevance (0 = only relevance, 1 = only diversity)

query_mmr = "Tell me about Air Canada policies and airport info."
mmr_results = mmr_retriever.invoke(query_mmr)
print(f"Query: '{query_mmr}'")
for i, doc in enumerate(mmr_results):
    print(f"Result {i+1} (MMR): {doc.page_content[:100]}... (Source: {doc.metadata.get('source')})")
print("\n")

--- Indexing Techniques & Retrieval Methods (Conceptual) ---
Vector databases like Chroma, Faiss, Pinecone, etc., internally use advanced indexing algorithms
such as HNSW (Hierarchical Navigable Small World) for fast Approximate Nearest Neighbor (ANN) search.
Compression techniques like Product Quantization (PQ) and Locality Sensitive Hashing (LSH)
are also handled internally by these databases to optimize storage and speed.
Retrieval primarily relies on similarity metrics like Cosine Similarity or Euclidean Distance.
When we call `retriever.invoke(query)` or `vectorstore.similarity_search()`, these algorithms are at work.

--- Maximal Marginal Relevance (MMR) Retrieval ---
Query: 'Tell me about Air Canada policies and airport info.'
Result 1 (MMR): To travel with a pet on Air Canada, it must be in an approved carrier, and fees apply. Check specifi... (Source: AirCanadaPolicy)
Result 2 (MMR): Loyalty programs like Aeroplan (Air Canada) offer points for flights and upgrades.... (Source:

3. Relevance Scoring in Hybrid Search using Reciprocal Rank Fusion (RRF)
(Already demonstrated in section 1 with reciprocal_rank_fusion function). RRF is excellent because it's score-agnostic, working well with different search algorithms that might return scores on different scales.

4. Using Auto-Cut Feature to Remove Irrelevant Results Dynamically
"Auto-cut" is typically a feature of vector databases that allows you to set a similarity score threshold, dynamically returning only results above that threshold. LangChain's retrievers can often pass score_threshold to the underlying vector store.

In [ ]:
# --- Auto-Cut / Score Threshold Filtering ---
print("--- Auto-Cut / Score Threshold Filtering ---")
# When using `similarity_search_with_score`, you can manually filter.
# Some retrievers/vector stores directly support `score_threshold`.
# FIX: Change search_type to "similarity_score_threshold" to enable score thresholding
retriever_with_threshold = flight_vectorstore.as_retriever(
    search_type="similarity_score_threshold", # Use the search type designed for score threshold
    search_kwargs={"k": 10, "score_threshold": 0.75} # Example threshold
)

query_autocut = "What kind of food do I get on a long flight?"
results_autocut = retriever_with_threshold.invoke(query_autocut)

print(f"Query: '{query_autocut}' (with score_threshold=0.75)")
if results_autocut:
    for i, doc in enumerate(results_autocut):
        # When using search_type="similarity_score_threshold", the results returned
        # by invoke are just the Document objects, not Document-score tuples.
        # The filtering by score is done internally.
        print(f"Result {i+1}: {doc.page_content[:100]}... (Metadata: {doc.metadata})")
else:
    print("No results found above the specified score threshold.")

# Manual filtering example using similarity_search_with_score
print("\n--- Manual Filtering with `similarity_search_with_score` ---")
min_acceptable_score = 0.7
manual_filtered_results = flight_vectorstore.similarity_search_with_score(query_autocut, k=10)
filtered = [(doc, score) for doc, score in manual_filtered_results if score >= min_acceptable_score]

if filtered:
    for i, (doc, score) in enumerate(filtered):
        print(f"Result {i+1} (score={score:.4f}): {doc.page_content[:100]}...")
else:
    print(f"No results found with similarity score >= {min_acceptable_score}.")
print("\n")

--- Auto-Cut / Score Threshold Filtering ---
Query: 'What kind of food do I get on a long flight?' (with score_threshold=0.75)
Result 1: American Airlines offers complimentary snacks on long-haul international flights.... (Metadata: {'airline': 'American Airlines', 'type': 'amenities', 'source': 'AmericanAirlines'})
Result 2: Economy class amenities on long international flights include meals and in-flight entertainment.... (Metadata: {'type': 'amenities', 'source': 'FlightExperience'})

--- Manual Filtering with `similarity_search_with_score` ---
No results found with similarity score >= 0.7.




5. Improving Search Relevance by using Language Understanding to Re-rank Search Results

Initial retrieval (e.g., from a vector database) is often based on embedding similarity. A re-ranking step uses a more powerful (often cross-encoder) model or even an LLM to evaluate the relevance of the retrieved documents to the query more deeply

In [ ]:
print("--- Re-ranking Retrieved Results ---")

# Step 1: Initial Retrieval (fetch more than needed for re-ranking)
query_rerank = "What do I need to know about carrying a cat on a flight?"
initial_retrieved_docs = flight_vectorstore.similarity_search(query_rerank, k=10) # Fetch top 10

print(f"Query for re-ranking: '{query_rerank}'")
print("Initial Retrieved Documents (before re-ranking):")
for i, doc in enumerate(initial_retrieved_docs):
    print(f"  {i+1}. {doc.page_content[:70]}...")

# Step 2: Use an LLM for re-ranking
# This is a simplified LLM re-ranker. Dedicated re-ranking models (e.g., Cohere Rerank) are more efficient.
# Here, we ask the LLM to score relevance for each document.

def llm_re_rank_documents(query: str, documents: List[str], llm_model: ChatOpenAI) -> List[Tuple[str, float]]:
    """
    Uses an LLM to assign a relevance score to each document relative to the query.
    Returns documents with their estimated scores.
    """
    re_rank_prompt = ChatPromptTemplate.from_messages([
        # Explicitly ask the LLM to return the exact document content string it received
        ("system", "You are a document relevance scorer. For each document provided, provide a relevance score between 0.0 (not relevant) and 1.0 (highly relevant) to the given query. Output a JSON array of objects, where each object has two keys: 'document' (the exact content of the document) and 'score' (the relevance score as a float). Only output JSON."),
        ("human", "Query: {query}\n\nDocuments:\n{documents}"),
    ])

    # Format documents for the LLM - send the content directly
    formatted_docs = "\n".join([f"--- Document {i+1} ---\n{doc}" for i, doc in enumerate(documents)])

    scoring_chain = re_rank_prompt | llm_model | JsonOutputParser()

    try:
        raw_scores = scoring_chain.invoke({"query": query, "documents": formatted_docs})
        # Assuming the LLM returns [{doc_content, score}, ...]
        # Map content back to original Document objects if available, or just keep content + score
        re_ranked_list = []
        for item in raw_scores:
            # Ensure 'document' key exists and is a string, and 'score' key exists and is a number
            if isinstance(item.get('document'), str) and isinstance(item.get('score'), (int, float)):
                original_doc_content = item['document']
                score = float(item['score'])
                re_ranked_list.append((original_doc_content, score))
            else:
                 print(f"Warning: Received unexpected item format from LLM: {item}")


        # Sort by score
        re_ranked_list.sort(key=lambda x: x[1], reverse=True)
        return re_ranked_list

    except Exception as e:
        print(f"Error during LLM re-ranking: {e}. Raw LLM output might be malformed JSON or unexpected data.")
        # Fallback: Return original documents with dummy scores or handle as error
        # Returning original documents with dummy scores maintains the structure for the next step
        return [(doc, 0.0) for doc in documents]


# Extract content strings for the LLM re-ranker
doc_contents = [doc.page_content for doc in initial_retrieved_docs]
re_ranked_docs_with_scores = llm_re_rank_documents(query_rerank, doc_contents, llm)

print("\nRe-ranked Documents (Top 5 based on LLM's assessment):")
# Ensure 'content' is indeed a string before slicing
for i, (content, score) in enumerate(re_ranked_docs_with_scores[:5]):
    # Add a check to ensure content is a string before slicing
    if isinstance(content, str):
        print(f"  {i+1}. {content[:70]}... (Score: {score:.4f})")
    else:
        # Handle cases where content is not a string, perhaps print a warning
        print(f"  {i+1}. [Error: Non-string content]... (Score: {score:.4f})")

print("\n")

--- Re-ranking Retrieved Results ---
Query for re-ranking: 'What do I need to know about carrying a cat on a flight?'
Initial Retrieved Documents (before re-ranking):
  1. To travel with a pet on Air Canada, it must be in an approved carrier,...
  2. Delta Airlines has a strict policy: carry-on must fit in the overhead ...
  3. Air Canada allows 1 carry-on bag (max 10kg) and 1 personal item. Check...
  4. Upgrading your seat on a flight often depends on availability and your...
  5. You can find your flight status on the airline's website or airport's ...
  6. When connecting through Toronto Pearson (YYZ), ensure you have enough ...
  7. American Airlines offers complimentary snacks on long-haul internation...
  8. Economy class amenities on long international flights include meals an...
  9. Business class passengers get priority boarding, lounge access, and li...
  10. Baggage policy for domestic flights with Air Canada: first checked bag...

Re-ranked Documents (Top 5 based on LLM's

6. Challenges: Scaling, Reliability, Cost Optimization

These are architectural considerations for deploying an AI agent.

* Scaling Optimization:
* * Vector Database: Use cloud-managed vector databases (Pinecone, Weaviate Cloud, Azure AI Search, AWS OpenSearch, Google Cloud Vertex AI Vector Search) that handle sharding, replication, and automatic scaling.

* * LLMs: Manage API rate limits, consider batching requests, or using larger models for complex tasks and smaller, faster models for simpler ones.

* * Orchestration: Use distributed computing frameworks (e.g., Kubernetes, serverless functions) for your LangChain application.


* Reliability Optimization:

* * Redundancy: Replicate vector databases and LLM services across regions/zones.

* * Monitoring: Implement robust logging and monitoring for LLM calls, tool executions, and retrieval performance.

* * Error Handling: Implement robust retry mechanisms, fallbacks (e.g., to simple keyword search if vector search fails), and circuit breakers.

* * Data Freshness: Strategies for keeping your vector store updated with the latest information (e.g., CDC - Change Data Capture, batch updates).

* Cost Optimization:

* * LLM Choice: Use smaller, cheaper LLMs for simpler tasks where possible. Optimize prompt length.

* * Embedding Models: Choose cost-effective embedding models.

* * Vector Database: Optimize indexing parameters (e.g., ef_construction, m in HNSW) for desired accuracy/speed vs. cost. Consider open-source self-hosted solutions for cost control (Chroma, Faiss) if you have the operational expertise.

* * Caching: Implement semantic caching to avoid redundant LLM calls (covered below).


7. Hands-on Exercises: Similarity Search, Hybrid Search, Vector Compression, Generative Search, Semantic Caching

* Similarity Search: (Already covered in section 1, flight_retriever.invoke(query_vector)).

* Hybrid Search: (Already covered in section 1 with reciprocal_rank_fusion).

* Vector Compression: (Conceptual, as it's internal to the vector DBs like PQ/LSH. Not directly exposed in LangChain at this level).

* Generative Search (RAG Pipeline): This combines retrieval with LLM generation to answer questions based on retrieved context.

In [ ]:
print("--- Generative Search (RAG Pipeline) ---")

rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant for flight planning. Use the following retrieved context to answer the user's question. If the context does not contain enough information, state that you cannot answer based on the provided context. Answer concisely and professionally."),
    ("user", "Context: {context}\n\nQuestion: {question}"),
])

# RAG Chain: retrieve -> format context -> LLM
rag_chain = (
    {"context": flight_retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

query_rag_1 = "What's the Air Canada policy on carry-on bags?"
print(f"Query (RAG): '{query_rag_1}'")
rag_response_1 = rag_chain.invoke(query_rag_1)
print(f"RAG Answer: {rag_response_1}\n")

query_rag_2 = "Tell me about the history of aviation." # This query should not find relevant docs
print(f"Query (RAG): '{query_rag_2}' (Should be out of context)")
rag_response_2 = rag_chain.invoke(query_rag_2)
print(f"RAG Answer: {rag_response_2}\n")

--- Generative Search (RAG Pipeline) ---
Query (RAG): 'What's the Air Canada policy on carry-on bags?'
RAG Answer: Air Canada's policy on carry-on bags allows for 1 carry-on bag with a maximum weight of 10 kg and 1 personal item.

Query (RAG): 'Tell me about the history of aviation.' (Should be out of context)
RAG Answer: I cannot answer based on the provided context.



* Semantic Caching:

* * Stores LLM responses based on the semantic similarity of the input query, not just exact string matching. This is done by embedding queries and comparing them. If a semantically similar query was asked before, the cached response can be returned, saving LLM costs and latency.

In [ ]:
print("--- Semantic Caching (Conceptual Implementation) ---")

# A simple dictionary-based semantic cache
class SemanticCache:
    def __init__(self, embedding_model, threshold=0.9):
        self.cache_store: Dict[str, Tuple[List[float], str]] = {} # {original_query: (embedding, response)}
        self.embedding_model = embedding_model
        self.threshold = threshold # Cosine similarity threshold for cache hit

    def _get_embedding(self, text: str) -> List[float]:
        return self.embedding_model.embed_query(text)

    def get(self, query: str) -> str | None:
        query_embedding = self._get_embedding(query)
        for original_query, (cached_embedding, response) in self.cache_store.items():
            similarity = np.dot(query_embedding, cached_embedding) / (np.linalg.norm(query_embedding) * np.linalg.norm(cached_embedding))
            if similarity >= self.threshold:
                print(f"[CACHE HIT] for '{query}' (matched '{original_query}', similarity: {similarity:.4f})")
                return response
        print(f"[CACHE MISS] for '{query}'")
        return None

    def set(self, query: str, response: str):
        query_embedding = self._get_embedding(query)
        self.cache_store[query] = (query_embedding, response)
        print(f"[CACHE SET] for '{query}'")

semantic_cache = SemanticCache(embeddings)

def cached_rag_chain_invoke(query: str, chain, cache: SemanticCache) -> str:
    cached_response = cache.get(query)
    if cached_response:
        return cached_response

    # If not in cache, invoke the RAG chain
    response = chain.invoke(query)
    cache.set(query, response)
    return response

print("\n--- Demonstrating Semantic Caching ---")

query_cache_1 = "What is the policy for carry-on luggage on Air Canada?"
print(f"Query 1: '{query_cache_1}'")
response_cache_1 = cached_rag_chain_invoke(query_cache_1, rag_chain, semantic_cache)
print(f"Response 1: {response_cache_1}\n")

query_cache_2 = "Tell me about Air Canada carry on bag limits." # Semantically similar
print(f"Query 2: '{query_cache_2}'")
response_cache_2 = cached_rag_chain_invoke(query_cache_2, rag_chain, semantic_cache)
print(f"Response 2: {response_cache_2}\n")

query_cache_3 = "How much does a checked bag cost on Air Canada domestic flights?" # Different query
print(f"Query 3: '{query_cache_3}'")
response_cache_3 = cached_rag_chain_invoke(query_cache_3, rag_chain, semantic_cache)
print(f"Response 3: {response_cache_3}\n")

query_cache_4 = "What are the rules for checked baggage on Air Canada internal flights?" # Semantically similar to query 3
print(f"Query 4: '{query_cache_4}'")
response_cache_4 = cached_rag_chain_invoke(query_cache_4, rag_chain, semantic_cache)
print(f"Response 4: {response_cache_4}\n")

# Clean up the ChromaDB directory
import shutil
if os.path.exists("./flight_docs_db"):
    shutil.rmtree("./flight_docs_db")
    print("Cleaned up ./flight_docs_db directory.")

--- Semantic Caching (Conceptual Implementation) ---

--- Demonstrating Semantic Caching ---
Query 1: 'What is the policy for carry-on luggage on Air Canada?'
[CACHE MISS] for 'What is the policy for carry-on luggage on Air Canada?'
[CACHE SET] for 'What is the policy for carry-on luggage on Air Canada?'
Response 1: Air Canada's policy for carry-on luggage allows 1 carry-on bag with a maximum weight of 10 kg and 1 personal item.

Query 2: 'Tell me about Air Canada carry on bag limits.'
[CACHE HIT] for 'Tell me about Air Canada carry on bag limits.' (matched 'What is the policy for carry-on luggage on Air Canada?', similarity: 0.9304)
Response 2: Air Canada's policy for carry-on luggage allows 1 carry-on bag with a maximum weight of 10 kg and 1 personal item.

Query 3: 'How much does a checked bag cost on Air Canada domestic flights?'
[CACHE MISS] for 'How much does a checked bag cost on Air Canada domestic flights?'
[CACHE SET] for 'How much does a checked bag cost on Air Canada domest

# chapter 3: Agentic Design Patterns

## Advanced Reasoning and Decision-Making with LLMs

Let's now move into the fascinating world of advanced reasoning and decision-making patterns for LLMs. This is where your flight planning AI agent truly gains intelligence, moving beyond simple question-answering to planning, executing, and even self-correcting its actions.

We'll leverage LangChain's powerful agent framework to implement these patterns.

Core Idea:

Instead of just responding directly to a prompt, an intelligent agent can:

* Reason (Thought): Internally deliberate, break down problems, and plan steps.
* Act (Action): Execute specific tools (APIs, functions, databases) to gather information or perform operations.
* Observe (Observation): Process the results of its actions.
* Reflect: Critically evaluate its past actions or current state and refine its strategy or output.

This iterative loop enables LLMs to tackle complex, multi-step problems that require external knowledge and dynamic interaction.

Prerequisites:

In [ ]:
!pip install langchain langchain-openai langchain_community -q

###  1. ReAct (Reason + Act) Framework
The ReAct framework combines reasoning (e.g., Thought in an LLM's output) with acting (using Action tools). The LLM iteratively thinks, chooses an action, observes the result, and then thinks again until it reaches a solution or answer.

Scenario: A flight booking agent needs to find flight information.

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.memory import ConversationBufferMemory
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field # Import BaseModel and Field

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# --- Define Tools for our ReAct Agent ---

class FlightInfo:
    def __init__(self, flight_id, origin, destination, departure_time, arrival_time, price, airline):
        self.flight_id = flight_id
        self.origin = origin
        self.destination = destination
        self.departure_time = departure_time
        self.arrival_time = arrival_time
        self.price = price
        self.airline = airline

    def to_string(self):
        return (f"Flight {self.flight_id} ({self.airline}): {self.origin} -> {self.destination}, "
                f"Dep: {self.departure_time}, Arr: {self.arrival_time}, Price: ${self.price}")

# Define Pydantic models for tool arguments
class SearchFlightsParams(BaseModel):
    origin: str = Field(description="The departure city or airport code.")
    destination: str = Field(description="The arrival city or airport code.")
    date: str = Field(description="The desired departure date in YYYY-MM-DD format.")

class GetAirportInfoParams(BaseModel):
    city: str = Field(description="The city for which to get airport information.")


@tool("search_flights", args_schema=SearchFlightsParams) # Use Pydantic model
def search_flights(origin: str, destination: str, date: str) -> List[dict]:
    """
    Searches for available flights between an origin and destination on a specific date.
    Returns a list of flight dictionaries, each with flight_id, origin, destination,
    departure_time, arrival_time, price, and airline.
    """
    print(f"\n--- Tool Call: search_flights({origin}, {destination}, {date}) ---")
    # Simulate flight data
    if "Montreal" in origin and "Paris" in destination and "2025-08-15" in date:
        return [
            {"flight_id": "AC123", "origin": origin, "destination": destination, "departure_time": "09:00", "arrival_time": "18:00", "price": 750, "airline": "Air Canada"},
            {"flight_id": "AF456", "origin": origin, "destination": destination, "departure_time": "10:30", "arrival_time": "19:30", "price": 820, "airline": "Air France"},
        ]
    elif "Tokyo" in origin and "London" in destination and "2025-09-01" in date:
         return [
            {"flight_id": "JL789", "origin": origin, "destination": destination, "departure_time": "14:00", "arrival_time": "22:00", "price": 1200, "airline": "JAL"},
        ]
    return []

@tool("get_airport_info", args_schema=GetAirportInfoParams) # Use Pydantic model
def get_airport_info(city: str) -> str:
    """
    Provides information about a specific city's major airport.
    """
    print(f"\n--- Tool Call: get_airport_info({city}) ---")
    if "Montreal" in city:
        return "Montreal's main airport is Pierre Elliott Trudeau International Airport (YUL)."
    elif "Paris" in city:
        return "Paris has two main international airports: Charles de Gaulle (CDG) and Orly (ORY)."
    return "Airport information not available for this city."

# All tools available to the agent
react_tools = [search_flights, get_airport_info]

# --- Create the ReAct Agent ---
# The prompt guides the LLM on how to reason and use tools.
react_agent_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI flight assistant. Use the provided tools to assist users with flight-related queries. Plan your steps and explain your reasoning."),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"), # This is where the Thought/Action/Observation history goes
])

# Create the agent
react_agent = create_tool_calling_agent(llm, react_tools, react_agent_prompt)

# Create the agent executor (the runtime for the agent)
react_agent_executor = AgentExecutor(agent=react_agent, tools=react_tools, verbose=True)


print("--- ReAct Agent Workflow ---")
print("User: Are there flights from Montreal to Paris on August 15, 2025?")
response_react_1 = react_agent_executor.invoke({"input": "Are there flights from Montreal to Paris on August 15, 2025?"})
print(f"Agent Response: {response_react_1['output']}\n")

print("User: What is the main airport in Paris?")
response_react_2 = react_agent_executor.invoke({"input": "What is the main airport in Paris?"})
print(f"Agent Response: {response_react_2['output']}\n")

--- ReAct Agent Workflow ---
User: Are there flights from Montreal to Paris on August 15, 2025?


> Entering new AgentExecutor chain...

Invoking: `search_flights` with `{'origin': 'Montreal', 'destination': 'Paris', 'date': '2025-08-15'}`



--- Tool Call: search_flights(Montreal, Paris, 2025-08-15) ---
[{'flight_id': 'AC123', 'origin': 'Montreal', 'destination': 'Paris', 'departure_time': '09:00', 'arrival_time': '18:00', 'price': 750, 'airline': 'Air Canada'}, {'flight_id': 'AF456', 'origin': 'Montreal', 'destination': 'Paris', 'departure_time': '10:30', 'arrival_time': '19:30', 'price': 820, 'airline': 'Air France'}]Yes, there are flights available from Montreal to Paris on August 15, 2025. Here are some options:

1. **Air Canada**
   - **Flight ID:** AC123
   - **Departure Time:** 09:00
   - **Arrival Time:** 18:00
   - **Price:** $750

2. **Air France**
   - **Flight ID:** AF456
   - **Departure Time:** 10:30
   - **Arrival Time:** 19:30
   - **Price:** $820

You can choose the f

Explanation:

When verbose=True, you'll see the "Thought, Action, Observation" loop.

* Thought: The LLM analyzes the user's query and decides which tool to use.
* Action: It calls a tool with specific arguments.
* Observation: It receives the output from the tool.
* This repeats until the LLM has enough information to formulate a final answer.

### 2. Reflection: Self-Checking and Improvement
Reflection allows the agent to evaluate its own reasoning or output and identify potential errors or areas for improvement. This often involves a meta-prompt or a "critic" component.

Scenario: After getting flight results, the agent might reflect if it has gathered enough information or if the results meet user's implicit needs (e.g., "Are these the cheapest/most convenient options?").

We'll implement a simple reflection step where the agent reviews its proposed final answer before presenting it.

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.memory import ConversationBufferMemory
from typing import List, Dict, Any, Optional
from pydantic import BaseModel, Field # Import BaseModel and Field
from langchain_core.output_parsers import StrOutputParser # Import StrOutputParser

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# --- Define Tools for our ReAct Agent ---

# Define Pydantic models for tool arguments

class SearchFlightsParams(BaseModel):
    origin: str = Field(description="The departure city or airport code.")
    destination: str = Field(description="The arrival city or airport code.")
    date: str = Field(description="The desired departure date in YYYY-MM-DD format.")

class GetAirportInfoParams(BaseModel):
    city: str = Field(description="The city for which to get airport information.")

# Define Pydantic model for reflect_on_answer tool arguments
class ReflectOnAnswerParams(BaseModel):
    original_query: str = Field(description="The user's original query.")
    proposed_answer: str = Field(description="The proposed answer generated by the agent.")

@tool("search_flights", args_schema=SearchFlightsParams) # Use Pydantic model
def search_flights(origin: str, destination: str, date: str) -> List[dict]:
    """
    Searches for available flights between an origin and destination on a specific date.
    Returns a list of flight dictionaries, each with flight_id, origin, destination,
    departure_time, arrival_time, price, and airline.
    """
    print(f"\n--- Tool Call: search_flights({origin}, {destination}, {date}) ---")
    # Simulate flight data
    if "Montreal" in origin and "Paris" in destination and "2025-08-15" in date:
        return [
            {"flight_id": "AC123", "origin": origin, "destination": destination, "departure_time": "09:00", "arrival_time": "18:00", "price": 750, "airline": "Air Canada"},
            {"flight_id": "AF456", "origin": origin, "destination": destination, "departure_time": "10:30", "arrival_time": "19:30", "price": 820, "airline": "Air France"},
        ]
    elif "Tokyo" in origin and "London" in destination and "2025-09-01" in date:
         return [
            {"flight_id": "JL789", "origin": origin, "destination": destination, "departure_time": "14:00", "arrival_time": "22:00", "price": 1200, "airline": "JAL"},
        ]
    return []

@tool("get_airport_info", args_schema=GetAirportInfoParams) # Use Pydantic model
def get_airport_info(city: str) -> str:
    """
    Provides information about a specific city's major airport.
    """
    print(f"\n--- Tool Call: get_airport_info({city}) ---")
    if "Montreal" in city:
        return "Montreal's main airport is Pierre Elliott Trudeau International Airport (YUL)."
    elif "Paris" in city:
        return "Paris has two main international airports: Charles de Gaulle (CDG) and Orly (ORY)."
    return "Airport information not available for this city."

# Define a 'reflect' tool that the agent can call
@tool("reflect_on_answer", args_schema=ReflectOnAnswerParams) # Use Pydantic model here
def reflect_on_answer(original_query: str, proposed_answer: str) -> str:
    """
    Critically evaluates a proposed answer based on the original query.
    Provides feedback on clarity, completeness, accuracy, and suggest improvements if any.
    Returns feedback as a string.
    """
    print(f"\n--- Tool Call: reflect_on_answer ---")
    reflection_prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a critical reviewer of AI agent responses. Your task is to evaluate a proposed answer against the original user query. Provide constructive feedback. Consider clarity, completeness, accuracy, and if the answer directly addresses the user's need. If the answer is perfect, say 'Perfect'. If not, suggest specific improvements."),
        ("human", "Original Query: {original_query}\n\nProposed Answer: {proposed_answer}\n\nFeedback:"),
    ])
    reflection_chain = reflection_prompt | llm | StrOutputParser()
    feedback = reflection_chain.invoke({"original_query": original_query, "proposed_answer": proposed_answer})
    return feedback


# All tools available to the agent
# react_tools already defined in the previous cell if running sequentially.
# Make sure react_tools is available or define it here if running this cell independently.
react_tools = [search_flights, get_airport_info]
reflection_tools = react_tools + [reflect_on_answer]


# --- Create the Reflection Agent ---
# Agent prompt for reflection. We need to guide it to use the reflection tool.
reflection_agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an AI flight assistant.
     First, use your tools to answer the user's question.
     After formulating a proposed final answer, use the 'reflect_on_answer' tool to get feedback on it.
     Then, if the feedback suggests improvements, try to incorporate them into your final response.
     If the feedback says 'Perfect', then output the proposed answer.
     Remember to keep track of the original query when reflecting.
     """),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

reflection_agent = create_tool_calling_agent(llm, reflection_tools, reflection_agent_prompt)
reflection_agent_executor = AgentExecutor(agent=reflection_agent, tools=reflection_tools, verbose=True)


print("--- Reflection Agent Workflow ---")

print("User: Find flights from Tokyo to London on Sep 1, 2025.")
response_reflection_1 = reflection_agent_executor.invoke({"input": "Find flights from Tokyo to London on Sep 1, 2025."})
print(f"Agent Response with Reflection: {response_reflection_1['output']}\n")

print("User: What is the airport in Montreal?")
response_reflection_2 = reflection_agent_executor.invoke({"input": "What is the airport in Montreal?"})
print(f"Agent Response with Reflection: {response_reflection_2['output']}\n")

--- Reflection Agent Workflow ---
User: Find flights from Tokyo to London on Sep 1, 2025.


> Entering new AgentExecutor chain...

Invoking: `search_flights` with `{'origin': 'Tokyo', 'destination': 'London', 'date': '2025-09-01'}`



--- Tool Call: search_flights(Tokyo, London, 2025-09-01) ---
[{'flight_id': 'JL789', 'origin': 'Tokyo', 'destination': 'London', 'departure_time': '14:00', 'arrival_time': '22:00', 'price': 1200, 'airline': 'JAL'}]
Invoking: `reflect_on_answer` with `{'original_query': 'Find flights from Tokyo to London on Sep 1, 2025.', 'proposed_answer': 'I found a flight from Tokyo to London on September 1, 2025. The flight is operated by JAL, with flight number JL789. It departs from Tokyo at 14:00 and arrives in London at 22:00. The price for this flight is $1200.'}`



--- Tool Call: reflect_on_answer ---
The proposed answer provides a specific flight option from Tokyo to London on the requested date, which is helpful. However, it could be improved by offering more 

Explanation:

The agent's prompt is modified to explicitly tell it to use the reflect_on_answer tool after it thinks it has a proposed answer. The reflect_on_answer tool itself is another LLM call with a "critic" persona. The agent then uses the Observation from this reflection to improve its final output.

### 3. CodeAct: Write + Execute Code Dynamically
CodeAct empowers LLMs to generate and execute code (like Python) to solve problems, especially those involving calculations, complex data manipulation, or logical reasoning that's hard to do purely with natural language.

Scenario: Calculate the layover time between two connecting flights or perform a currency conversion for flight prices.

In [ ]:
!pip install langchain_experimental -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 3.8 MB/s eta 0:00:00


In [ ]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults # For web search if needed
from langchain_experimental.tools.python.tool import PythonREPLTool
from datetime import datetime, timedelta

print("--- CodeAct Agent Workflow ---")

# Python REPL tool allows the agent to execute Python code
python_repl = PythonREPLTool()

# Extend tools to include the Python REPL
codeact_tools = react_tools + [python_repl]

codeact_agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an intelligent assistant. You have access to tools, including a Python REPL.
     If you need to perform calculations, data manipulation, or complex logic, you can write and execute Python code.
     The output of the Python code will be in the 'Observation' step.
     Use your tools to answer flight-related queries.
     Today's date is 2025-05-26.
     """),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

codeact_agent = create_tool_calling_agent(llm, codeact_tools, codeact_agent_prompt)
codeact_agent_executor = AgentExecutor(agent=codeact_agent, tools=codeact_tools, verbose=True)

print("User: If I arrive in Toronto at 14:30 on 2025-07-01 and my connecting flight departs at 18:45 on the same day, how long is my layover?")
response_codeact_1 = codeact_agent_executor.invoke({"input": "If I arrive in Toronto at 14:30 on 2025-07-01 and my connecting flight departs at 18:45 on the same day, how long is my layover?"})
print(f"Agent Response (CodeAct): {response_codeact_1['output']}\n")

print("User: I found a flight for $820 USD. If the exchange rate is 1 USD = 1.37 CAD, how much is it in Canadian Dollars?")
response_codeact_2 = codeact_agent_executor.invoke({"input": "I found a flight for $820 USD. If the exchange rate is 1 USD = 1.37 CAD, how much is it in Canadian Dollars?"})
print(f"Agent Response (CodeAct): {response_codeact_2['output']}\n")

# A more complex scenario: find flights and then calculate potential layover with dummy data
print("User: Find flights from Montreal to Paris on August 15, 2025. Also, if a flight from Montreal to Paris arrives at 18:00 and the next flight departs at 20:30, calculate the layover.")
response_codeact_3 = codeact_agent_executor.invoke({"input": "Find flights from Montreal to Paris on August 15, 2025. Also, if a flight from Montreal to Paris arrives at 18:00 and the next flight departs at 20:30, calculate the layover."})
print(f"Agent Response (CodeAct - combined): {response_codeact_3['output']}\n")

--- CodeAct Agent Workflow ---
User: If I arrive in Toronto at 14:30 on 2025-07-01 and my connecting flight departs at 18:45 on the same day, how long is my layover?


> Entering new AgentExecutor chain...



Invoking: `Python_REPL` with `{'query': "from datetime import datetime\n\narrival_time = datetime.strptime('2025-07-01 14:30', '%Y-%m-%d %H:%M')\ndeparture_time = datetime.strptime('2025-07-01 18:45', '%Y-%m-%d %H:%M')\n\nlayover_duration = departure_time - arrival_time\n\nhours, remainder = divmod(layover_duration.seconds, 3600)\nminutes = remainder // 60\n\n(hours, minutes)"}`




> Finished chain.
Agent Response (CodeAct): 

User: I found a flight for $820 USD. If the exchange rate is 1 USD = 1.37 CAD, how much is it in Canadian Dollars?


> Entering new AgentExecutor chain...

Invoking: `Python_REPL` with `{'query': 'usd_price = 820\nexchange_rate = 1.37\ncad_price = usd_price * exchange_rate\ncad_price'}`




> Finished chain.
Agent Response (CodeAct): 

User: Find flights from Montreal to Paris on August 15, 2025. Also, if a flight from Montreal to Paris arrives at 18:00 and the next flight departs at 20:30, calculate the layover.


> Entering new AgentExecutor chain...

Invokin

Explanation:

The agent is now equipped with PythonREPLTool. When it needs to do calculations, it will output Action: python_repl followed by the Python code, execute it, and then observe the result. This enables dynamic computation.

### 4. Combining Patterns into Flexible Agents
You can combine these patterns to create highly flexible and robust agents. For instance:

* A ReAct agent could incorporate Reflection to ensure its answers are robust.
* A CodeAct agent could use Reflection to check the correctness or efficiency of the code it generates before execution.
* A research agent might use ReAct to search for information, then CodeAct to analyze the retrieved data, and finally Reflection to synthesize and verify its findings.

The LangChain AgentExecutor allows you to simply pass in a list of tools (including the PythonREPLTool and your custom reflection tools). The agent's prompt then becomes crucial for guiding it on when and how to use these diverse capabilities.

Conceptual Example of a Combined Agent:

In [ ]:
print("--- Combining Patterns (Conceptual) ---")

# All tools combined
combined_tools = react_tools + [reflect_on_answer, python_repl]

combined_agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are an advanced AI flight planning assistant.
     Your capabilities include:
     1.  **Searching Flights & Airports:** Use 'search_flights' and 'get_airport_info'.
     2.  **Calculations:** Use the 'python_repl' tool for any numerical computations (e.g., layover times, currency conversion).
     3.  **Self-Correction (Reflection):** After you formulate a proposed answer or plan, use 'reflect_on_answer' to critically review it for clarity, completeness, and accuracy. Incorporate feedback.

     Always strive for the most accurate and helpful answer. Plan your steps.
     Today's date is 2025-05-26.
     """),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

combined_agent = create_tool_calling_agent(llm, combined_tools, combined_agent_prompt)
combined_agent_executor = AgentExecutor(agent=combined_agent, tools=combined_tools, verbose=True)

print("User: I need flights from Montreal to Paris on Aug 15, 2025. After you find them, tell me the layover if the Air Canada flight arrives at 18:00 and my next flight departs at 20:00. Then reflect on your answer.")
response_combined = combined_agent_executor.invoke({"input": "I need flights from Montreal to Paris on Aug 15, 2025. After you find them, tell me the layover if the Air Canada flight arrives at 18:00 and my next flight departs at 20:00. Then reflect on your answer."})
print(f"Agent Response (Combined): {response_combined['output']}\n")

--- Combining Patterns (Conceptual) ---
User: I need flights from Montreal to Paris on Aug 15, 2025. After you find them, tell me the layover if the Air Canada flight arrives at 18:00 and my next flight departs at 20:00. Then reflect on your answer.


> Entering new AgentExecutor chain...

Invoking: `search_flights` with `{'origin': 'Montreal', 'destination': 'Paris', 'date': '2025-08-15'}`



--- Tool Call: search_flights(Montreal, Paris, 2025-08-15) ---
[{'flight_id': 'AC123', 'origin': 'Montreal', 'destination': 'Paris', 'departure_time': '09:00', 'arrival_time': '18:00', 'price': 750, 'airline': 'Air Canada'}, {'flight_id': 'AF456', 'origin': 'Montreal', 'destination': 'Paris', 'departure_time': '10:30', 'arrival_time': '19:30', 'price': 820, 'airline': 'Air France'}]
Invoking: `Python_REPL` with `{'query': "from datetime import datetime\narrival_time = datetime.strptime('18:00', '%H:%M')\ndeparture_time = datetime.strptime('20:00', '%H:%M')\nlayover_duration = departure_time - arr

Key Takeaway for Combination:

The complexity shifts from explicit code for each step to prompt engineering. You guide the LLM to understand its diverse capabilities and when to apply them based on the user's intent. The verbose=True output becomes invaluable for debugging and understanding the agent's internal thought process.

### 5. Use Cases:
These agentic patterns are broadly applicable:

* Flight Planning Agent (Our Goal):

* * ReAct: For finding flights, getting airport info, checking policies.

* * Reflection: To verify if the proposed flight options meet all user constraints or if a booking confirmation is clear.

* * CodeAct: For calculating best routes based on complex criteria, comparing prices after currency conversion, calculating total travel time including layovers, or dynamically generating specific data reports.

* Coding Agents: Generating code, testing it (CodeAct), identifying bugs, and refactoring (Reflection).

* Research Agents: Decomposing research questions, performing multi-step searches (ReAct), synthesizing information, identifying gaps, and refining hypotheses (Reflection).

* Evaluators: Critiquing outputs from other LLMs or systems (Reflection), or generating test cases (CodeAct).

By implementing these advanced reasoning and decision-making patterns, you are significantly enhancing the intelligence and autonomy of your flight planning AI agent, allowing it to handle more complex and dynamic user requests.

# chapter 4: Interoperability of Agents

Now, let's explore how to make your flight planning AI agent interoperable, allowing it to communicate and collaborate with other agents, tools, and APIs across diverse platforms. This is essential for building complex AI ecosystems where specialized agents can work together.

Interoperability: Agents Calling and Responding to Each Other

## Google A2A (Agents-to-Agents) Overview:

Based on recent information, the Agent2Agent (A2A) Protocol is an open standard, developed by Google, designed to enable seamless communication and collaboration between AI agents built using diverse frameworks and by different vendors. It aims to break down silos and foster interoperability, acting like "APIs but for agent communication."


Key aspects of A2A:

* Universal Communication Standard: Allows agents from different ecosystems (e.g., Google's ADK, LangChain, Crew.ai) to communicate.
Dynamic Discovery: Agents can dynamically discover each other's capabilities.

* Negotiation: Agents can negotiate interaction formats (text, forms, audio/video).

* Multi-modal Content: Supports sharing not just text, but also files, structured data, and potentially richer media.

* Long-running Processes: Designed to handle complex, multi-step workflows that might take hours or involve human input.

* Security: Incorporates enterprise-grade security mechanisms.

* Streamlined Integration: Reduces the need for custom "glue code" for every agent-to-agent interaction.

For our flight planning agent, A2A would mean it could seamlessly interact with a separate "Hotel Booking Agent," a "Weather Agent," or a "Payment Processing Agent," all operating independently but collaborating on a user's travel plan.

Cross-Agent Communication Architecture:

Implementing cross-agent communication typically involves:

1. Modular Agent Design: Each agent is a specialized, independent entity with clear boundaries and well-defined input/output interfaces.

2. Standardized Communication Protocols: Agents exchange messages using agreed-upon formats and protocols (like A2A, or simpler HTTP/REST APIs, messaging queues).

3. Discovery Mechanisms: Agents need a way to find and understand the capabilities of other agents (e.g., a central registry, dynamic advertisement).

4. Orchestration Layer: A higher-level component (or a lead agent) that coordinates interactions between multiple specialized agents.

5. Context Management: Mechanisms to maintain conversational context and state across agents.

6. Security and Authentication: Ensuring secure and authorized communication between agents.

Creating API-Ready Agents:

To enable agents to call each other, they need to expose their capabilities via APIs.

* Clear Functionality: Define precise actions and data inputs/outputs for each agent's API endpoint.

* Structured Inputs/Outputs: Use standard data formats like JSON.

* Robust Documentation: Provide clear, LLM-consumable documentation for API endpoints and parameters (e.g., using OpenAPI/Swagger specs).

* Error Handling: Implement clear and informative error messages.

* Authentication/Authorization: Secure API access (e.g., OAuth, API keys). OAuth is recommended for delegating access to user resources.

* Granularity and Pagination: Design APIs to allow granular data retrieval and handle large datasets via pagination.

Token Hand-off Strategies Across Multiple LLMs:

When multiple LLMs (or agents powered by different LLMs) interact, managing context (tokens) is crucial for efficiency and coherence.

1. Context Summarization: A preceding LLM summarizes the relevant parts of the conversation/data before passing it to the next LLM.

2. Shared Memory/Database: Agents store relevant context (e.g., user preferences, extracted entities, previous API results) in a shared, structured memory (like a vector database or a traditional database) that subsequent agents can query.

3. Token Exchange (OAuth): For authentication, an agent might obtain an access token and exchange it with another trusted service to gain access without re-authenticating the user for every service.

4. Entity Extraction & Hand-off: Instead of raw text, extract key entities (flight details, dates, passenger names) and hand over these structured entities, which are more token-efficient.

5. Prompt Engineering for Contextual Chains: Design prompts that explicitly instruct LLMs on how to consume context from previous steps/agents.

Building Language-Agnostic Agent Endpoints:

For broad interoperability, agent endpoints should be language-agnostic.

* RESTful APIs with JSON: This is the most common and universally understood approach. Any programming language can make HTTP requests and parse JSON.

* Standardized Schemas (OpenAPI/JSON Schema): Define API contracts using OpenAPI Specification or JSON Schema. This allows tools and agents to automatically understand how to interact with the endpoint, regardless of the underlying implementation language.

* Semantic Interoperability: Ensure not just syntactic compatibility but also semantic understanding. This means that when one agent sends "origin: Montreal," the receiving agent correctly understands "Montreal" as a departure city. This often relies on shared ontologies or explicit schema definitions.

* Idempotency: Design API calls to be idempotent where applicable, meaning multiple identical requests have the same effect as a single request.

Hands-on: Deploy Agents that Call and Respond to Each Other
For a practical "hands-on" example, we'll simulate cross-agent communication using Python functions that act as API endpoints for simplicity. In a real-world scenario, these "agents" would likely be separate microservices deployed on a platform like Flask/FastAPI, accessible via HTTP.

We'll create two agents:

1. FlightSearchAgent: Specializes in finding flights.

2. HotelBookingAgent: Specializes in finding hotels.

Our main user-facing agent (the "Orchestrator") will then call these specialized agents as if they were external services.

Prerequisites (if not already installed):

In [ ]:
!pip install langchain langchain-openai pydantic -q

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain.memory import ConversationBufferMemory
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional
import json

# Initialize LLM
llm = ChatOpenAI(model="gpt-4o", temperature=0) # Using gpt-4o as a powerful model

# --- Define Pydantic Models for Tool Arguments ---

class FlightSearchParams(BaseModel):
    """Parameters for searching flights."""
    origin: str = Field(description="The departure city or airport code.")
    destination: str = Field(description="The arrival city or airport code.")
    date: str = Field(description="The desired departure date in YYYY-MM-DD format.")

class HotelSearchParams(BaseModel):
    """Parameters for searching hotels."""
    location: str = Field(description="The city or area for the hotel search.")
    check_in_date: str = Field(description="The desired check-in date in YYYY-MM-DD format.")
    check_out_date: str = Field(description="The desired check-out date in YYYY-MM-DD format.")
    num_guests: int = Field(description="Number of guests for the hotel booking.")

# --- Define Tool Implementations (Define BEFORE classes that use them) ---

# Note: The implementations themselves don't change, they just expect string/int arguments.
# The Pydantic model ensures these arguments are correctly parsed from the LLM's output.

def _search_flights_tool_impl(origin: str, destination: str, date: str) -> List[dict]:
    """
    Implementation of the API to search for available flights.
    Returns a list of simulated flight dictionaries.
    """
    print(f"  [FlightSearchAgent] Calling external Flight API for: {origin} -> {destination} on {date}")
    # Simulate an external API call
    if "Montreal" in origin and "Paris" in destination and "2025-08-15" in date:
        return [
            {"flight_id": "AC123", "origin": origin, "destination": destination, "departure_time": "09:00", "arrival_time": "18:00", "price": 750, "airline": "Air Canada"},
            {"flight_id": "AF456", "origin": origin, "destination": destination, "departure_time": "10:30", "arrival_time": "19:30", "price": 820, "airline": "Air France"},
        ]
    return []

def _Google_Hotels_tool_impl(location: str, check_in_date: str, check_out_date: str, num_guests: int) -> List[dict]:
    """
    Implementation of the API to search for hotels.
    Returns a list of simulated hotel dictionaries.
    """
    print(f"  [HotelBookingAgent] Calling external Hotel API for: {location} ({num_guests} guests) from {check_in_date} to {check_out_date}")
    # Simulate an external API call
    if "Paris" in location and "2025-08-15" in check_in_date:
        return [
            {"hotel_id": "HP789", "name": "Hotel Paradis", "location": location, "price_per_night": 200, "stars": 4, "availability": True},
            {"hotel_id": "RS012", "name": "Riverside Suites", "location": location, "price_per_night": 150, "stars": 3, "availability": True},
            ]
    return [] # Added return for the case where the location/date don't match

# --- 1. Define Specialized Agents (Simulated as Python Classes) ---
# These classes now reference the functions defined above and the Pydantic models

class FlightSearchAgent:
    """
    A specialized agent that handles flight searching.
    Its capabilities are exposed via a tool.
    """
    def __init__(self, llm):
        self.llm = llm
        # Define the tool using the external implementation function and the Pydantic model
        self._search_flights_tool_bound = tool("search_flights_api", args_schema=FlightSearchParams)(_search_flights_tool_impl)
        self.tools = [self._search_flights_tool_bound]
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful flight search assistant. Use the 'search_flights_api' tool to find flights. Extract origin, destination, and date clearly."),
            ("user", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ])
        # Use create_tool_calling_agent with the bound tool
        self.agent = create_tool_calling_agent(self.llm, self.tools, self.prompt)
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, verbose=True) # Set to True for inner agent logs

    def run(self, query: str) -> str:
        """Exposes the agent's functionality as a simple run method (simulating an API endpoint)."""
        print(f"  [FlightSearchAgent] Received query: '{query}'")
        # Call the inner agent executor
        result = self.executor.invoke({"input": query})
        # Check if the result has the expected 'output' key
        return result.get('output', 'No output generated by inner agent.')


class HotelBookingAgent:
    """
    A specialized agent that handles hotel booking.
    Its capabilities are exposed via a tool.
    """
    def __init__(self, llm):
        self.llm = llm
        # Define the tool using the external implementation function and the Pydantic model
        self._Google_Hotels_tool_bound = tool("Google_Hotels_api", args_schema=HotelSearchParams)(_Google_Hotels_tool_impl)
        # Renamed tool to remove space as it can cause issues
        self.tools = [self._Google_Hotels_tool_bound]
        self.prompt = ChatPromptTemplate.from_messages([
            # Updated prompt to reflect the corrected tool name
            ("system", "You are a helpful hotel search assistant. Use the 'Google_Hotels_api' tool to find hotels. Extract location, check-in, check-out dates, and number of guests clearly."),
            ("user", "{input}"),
            ("placeholder", "{agent_scratchpad}"),
        ])
        # Use create_tool_calling_agent with the bound tool
        self.agent = create_tool_calling_agent(self.llm, self.tools, self.prompt)
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, verbose=True) # Set to True for inner agent logs

    def run(self, query: str) -> str:
        """Exposes the agent's functionality as a simple run method (simulating an API endpoint)."""
        print(f"  [HotelBookingAgent] Received query: '{query}'")
        # Call the inner agent executor
        result = self.executor.invoke({"input": query})
        # Check if the result has the expected 'output' key
        return result.get('output', 'No output generated by inner agent.')


# --- Instantiate our specialized agents (Define AFTER classes) ---
# Passing the LLM instance to the agents
flight_agent_instance = FlightSearchAgent(llm)
hotel_agent_instance = HotelBookingAgent(llm)

# --- 2. Create API-Ready Proxies (Tools for Orchestrator Agent) ---
# These tool definitions reference the instantiated agents (Define AFTER agents instantiated)

class AgentQuery(BaseModel):
    """Parameters for calling a specialized agent."""
    query: str = Field(description="The specific query to pass to the specialized agent.")

@tool("call_flight_search_agent", args_schema=AgentQuery)
def call_flight_search_agent(query: str) -> str:
    """
    Calls the Flight Search Agent to find flight information.
    Passes the relevant query to the flight agent and returns its response.
    """
    print(f"\n[Orchestrator] Calling FlightSearchAgent with query: '{query}'")
    # Delegate the query to the specialized agent's run method
    return flight_agent_instance.run(query)

# The HotelBookingAgent tool also needs a Pydantic args_schema
@tool("call_hotel_booking_agent", args_schema=AgentQuery)
def call_hotel_booking_agent(query: str) -> str:
    """
    Calls the Hotel Booking Agent to find hotel information.
    Passes the relevant query to the hotel agent and returns its response.
    """
    print(f"\n[Orchestrator] Calling HotelBookingAgent with query: '{query}')")
    # Delegate the query to the specialized agent's run method
    return hotel_agent_instance.run(query)

# --- 3. Build the Orchestrator Agent (Define AFTER orchestrator tools) ---

orchestrator_tools = [call_flight_search_agent, call_hotel_booking_agent]

orchestrator_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a highly capable travel planning orchestrator AI.
     Your task is to understand the user's request and delegate to the appropriate specialized agent (FlightSearchAgent or HotelBookingAgent) using their respective tools.
     If the user asks for flights, use the 'call_flight_search_agent' tool.
     If the user asks for hotels, use the 'call_hotel_booking_agent' tool.
     If the user asks for both, call both agents sequentially or in parallel if needed to gather all information before responding comprehensively.
     Always provide a clear, concise answer to the user based on the specialized agents' responses.
     """),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

# Import the necessary modules explicitly here as well, just in case.
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory

orchestrator_agent = create_tool_calling_agent(llm, orchestrator_tools, orchestrator_prompt)
# The memory is currently not used in the orchestrator_agent_executor in this cell's example usage.
# If memory is needed for the orchestrator, it should be passed here.
# For this specific error, memory is not the direct cause, but keep it in mind for future statefulness.
orchestrator_agent_executor = AgentExecutor(agent=orchestrator_agent, tools=orchestrator_tools, verbose=True)

print("--- Orchestrator Agent in Action (Agent-to-Agent Communication) ---")

# --- Hands-on Exercise 1: Orchestrator calls Flight Agent ---
print("\nUser: Find me flights from Montreal to Paris on August 15, 2025.")
# The invoke method expects a dictionary input matching the agent's prompt variables.
# In this case, the prompt only has 'input'.
orchestrator_response_1 = orchestrator_agent_executor.invoke({"input": "Find me flights from Montreal to Paris on August 15, 2025."})
print(f"\nFINAL ORCHESTRATOR RESPONSE: {orchestrator_response_1['output']}\n")

# --- Hands-on Exercise 2: Orchestrator calls Hotel Agent ---
print("\nUser: I need a hotel in Paris for August 15 to August 20, 2025 for 2 guests.")
orchestrator_response_2 = orchestrator_agent_executor.invoke({"input": "I need a hotel in Paris for August 15 to August 20, 2025 for 2 guests."})
print(f"\nFINAL ORCHESTRATOR RESPONSE: {orchestrator_response_2['output']}\n")

# --- Hands-on Exercise 3: Orchestrator calls both agents (multi-turn or combined) ---
print("\nUser: Plan a trip for me: flights from Montreal to Paris on Aug 15, 2025, AND a hotel in Paris from Aug 15 to Aug 20, 2025 for 2 people.")
orchestrator_response_3 = orchestrator_agent_executor.invoke({"input": "Plan a trip for me: flights from Montreal to Paris on Aug 15, 2025, AND a hotel in Paris from Aug 15 to Aug 20, 2025 for 2 people."})
print(f"\nFINAL ORCHESTRATOR RESPONSE: {orchestrator_response_3['output']}\n")

--- Orchestrator Agent in Action (Agent-to-Agent Communication) ---

User: Find me flights from Montreal to Paris on August 15, 2025.


> Entering new AgentExecutor chain...

Invoking: `call_flight_search_agent` with `{'query': 'Find flights from Montreal to Paris on August 15, 2025.'}`



[Orchestrator] Calling FlightSearchAgent with query: 'Find flights from Montreal to Paris on August 15, 2025.'
  [FlightSearchAgent] Received query: 'Find flights from Montreal to Paris on August 15, 2025.'


> Entering new AgentExecutor chain...

Invoking: `search_flights_api` with `{'origin': 'Montreal', 'destination': 'Paris', 'date': '2025-08-15'}`


  [FlightSearchAgent] Calling external Flight API for: Montreal -> Paris on 2025-08-15
[{'flight_id': 'AC123', 'origin': 'Montreal', 'destination': 'Paris', 'departure_time': '09:00', 'arrival_time': '18:00', 'price': 750, 'airline': 'Air Canada'}, {'flight_id': 'AF456', 'origin': 'Montreal', 'destination': 'Paris', 'departure_time': '10:30', 'arriva

Explanation and Key Learnings:

1. Specialized Agents (FlightSearchAgent, HotelBookingAgent):

* *  Each is an independent AgentExecutor with its own llm, tools, and prompt tailored to its specific domain.

* * Their run() method simulates an API endpoint, taking a query and returning a response.

* * They internally use their own specialized tools (e.g., _search_flights_tool) which represent calls to actual external APIs.

2. API-Ready Proxies (Tools for Orchestrator):

* * call_flight_search_agent and call_hotel_booking_agent are LangChain tools for the orchestrator_agent.

* * Crucially, the implementation of these tools is to call the run() method of the respective specialized agent instances. This simulates one agent making an API call to another agent.

3. Orchestrator Agent:

* * This is the user-facing agent. Its prompt guides it to understand user intent and choose which specialized agent (via its proxy tool) to invoke.

* * It doesn't directly call the external Flight or Hotel APIs; it delegates to the specialized agents.

3. Cross-Agent Communication Architecture:

* * This setup demonstrates a hierarchical multi-agent system. The orchestrator acts as a coordinator, delegating tasks to expert agents.

* * Context Hand-off: The orchestrator extracts relevant parts of the user's query and passes it as a new query to the specialized agent. The specialized agent then processes this new context.
For more complex hand-offs, you'd serialize memory, specific extracted entities, or summarized conversation history.

* * Language-Agnostic Endpoints (Conceptual): While our example uses Python function calls, in a real system, FlightSearchAgent.run() and HotelBookingAgent.run() would typically be exposed via HTTP endpoints (e.g., using Flask/FastAPI).
The call_..._agent tools would then make HTTP requests to these endpoints, making the underlying implementation language irrelevant to the caller.


This architecture provides a powerful way to build complex AI systems by breaking them down into manageable, interoperable components, much like microservices. It's a foundational step towards building the comprehensive and intelligent flight planning AI agent we envision.








# chapter 5: Observability and Monitoring

Establishing robust monitoring for your LLM agents is critical for understanding their behavior, debugging complex workflows, ensuring safety, and maintaining reliability in production. This becomes even more vital for an AI flight planning agent where correctness and responsiveness are paramount.

Robust Monitoring for LLM Agents
Key Concepts:

1. Logging and Tracing Agent Decisions:

* * Logging: Recording discrete events (e.g., agent starts, tool calls, LLM calls, final answers, errors) to files or a logging system.

* * Tracing: Capturing the entire execution path of an agent, including all its internal thoughts, actions, observations, and tool calls, providing a chronological flow. This is crucial for understanding why an agent made a particular decision.

2. Callback Mechanisms in LangChain & LangGraph:

* * LangChain (and LangGraph) provides a powerful Callback system. This allows you to "hook into" various stages of an LLM chain or agent execution.

* * You can define custom callback handlers that execute specific code (e.g., logging to a database, sending metrics to a monitoring system, printing to console) whenever an event occurs (e.g., LLM starts/ends, tool starts/ends, chain starts/ends).

3. Tracking Token Usage, Latency, Success Rate:

* * Token Usage: Monitor input/output tokens for LLM calls to manage costs and evaluate efficiency.

* * Latency: Measure the time taken for LLM calls, tool executions, and overall agent responses to identify bottlenecks.


* * Success Rate: Track how often agents successfully complete tasks, use tools correctly, or return valid responses. This often requires defining what constitutes "success" (e.g., "tool call was successful," "output was parseable," "user query was answered").

4. Visual Debugging of Agent Flows:

* * Tools like LangSmith (from LangChain's creators) provide a visual interface to trace agent runs. You can see each thought, action, observation, LLM input/output, and tool output in a clear, nested waterfall diagram. This is invaluable for debugging complex multi-step reasoning.

* * Other platforms like Datadog, Grafana, or custom dashboards can visualize aggregated metrics.


Hands-on Exercise: Add Observability to Your Agent Workflow
We'll use a simplified version of our orchestrator_agent and its sub-agents from the previous example. We'll implement a custom BaseCallbackHandler to capture key events and metrics.

Prerequisites:

In [ ]:
!pip install langchain langchain-openai pydantic -q

In [ ]:
import os
import time
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from pydantic import BaseModel, Field
from typing import List, Dict, Any, Optional
from langchain_core.callbacks import BaseCallbackHandler # Crucial for observability
from uuid import UUID # For tracking run IDs
from langchain_core.messages import BaseMessage
from langchain_core.outputs import LLMResult

# --- 1. Initialize LLM (same as before) ---
# Re-instantiate after library upgrade
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# --- 2. Re-define Specialized Agents (adapted to accept callbacks) ---
# We make sure their `run` methods can accept and pass down callbacks.
class FlightSearchAgent:
    def __init__(self, llm_instance):
        self.llm = llm_instance
        # Define Pydantic model for the tool's arguments
        class FlightSearchInput(BaseModel):
            origin: str = Field(description="The origin city or airport code.")
            destination: str = Field(description="The destination city or airport code.")
            date: str = Field(description="The desired date of travel in YYYY-MM-DD format.")

        @tool("search_flights_api", args_schema=FlightSearchInput) # Use the Pydantic model
        def _search_flights_tool(origin: str, destination: str, date: str) -> List[dict]:
            """Searches for flights between origin and destination on a specific date.""" # Added docstring
            print(f"  [FlightSearchAgent] Simulating Flight API call for: {origin} -> {destination} on {date}")
            time.sleep(0.5) # Simulate latency
            if "Montreal" in origin and "Paris" in destination and "2025-08-15" in date:
                return [{"flight_id": "AC123", "price": 750}, {"flight_id": "AF456", "price": 820}]
            return []
        self.tools = [_search_flights_tool]
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful flight search assistant. Use the 'search_flights_api' tool."),
            ("user", "{input}"),
            ("placeholder", "{agent_scratchpad}")
        ])
        self.agent = create_tool_calling_agent(self.llm, self.tools, self.prompt)
        # Internal verbose can be off, as we'll use callbacks for structured logging
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, verbose=False)

    def run(self, query: str, callbacks=None) -> str:
        # Pass callbacks to the inner executor if provided
        result = self.executor.invoke({"input": query}, config={"callbacks": callbacks})
        return result['output']

class HotelBookingAgent:
    def __init__(self, llm_instance):
        self.llm = llm_instance
        # Define Pydantic model for the tool's arguments
        class HotelSearchInput(BaseModel):
            location: str = Field(description="The location for the hotel search.")
            check_in_date: str = Field(description="The check-in date in YYYY-MM-DD format.")
            check_out_date: str = Field(description="The check-out date in YYYY-MM-DD format.")
            num_guests: int = Field(description="The number of guests.")

        @tool("Google_Hotels_api", args_schema=HotelSearchInput) # Use the Pydantic model
        def _Google_Hotels_tool(location: str, check_in_date: str, check_out_date: str, num_guests: int) -> List[dict]:
            """Searches for hotels in a location for specified dates and number of guests.""" # Added docstring
            print(f"  [HotelBookingAgent] Simulating Hotel API call for: {location} for {num_guests} guests")
            time.sleep(0.3) # Simulate latency
            if "Paris" in location and "2025-08-15" in check_in_date:
                return [{"hotel_id": "HP789", "price_per_night": 200}, {"hotel_id": "RS012", "price_per_night": 150}]
            return []
        self.tools = [_Google_Hotels_tool]
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful hotel search assistant. Use the 'Google_Hotels_api' tool."),
            ("user", "{input}"),
            ("placeholder", "{agent_scratchpad}")
        ])
        self.agent = create_tool_calling_agent(self.llm, self.tools, self.prompt)
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, verbose=False)

    def run(self, query: str, callbacks=None) -> str:
        result = self.executor.invoke({"input": query}, config={"callbacks": callbacks})
        return result['output']


# Define Pydantic models for the orchestrator's tool inputs
class FlightSearchOrchestratorInput(BaseModel):
    query: str = Field(description="The user's query regarding flight search.")

class HotelBookingOrchestratorInput(BaseModel):
    query: str = Field(description="The user's query regarding hotel search.")


# Instantiate specialized agents
flight_agent_instance = FlightSearchAgent(llm)
hotel_agent_instance = HotelBookingAgent(llm)

# --- 3. Redefine API-ready Proxies to pass callbacks ---
# Crucially, tools are now explicitly told to pass callbacks.
# Use the Pydantic model for args_schema
@tool("call_flight_search_agent", args_schema=FlightSearchOrchestratorInput)
def call_flight_search_agent(query: str, callbacks: Optional[List[BaseCallbackHandler]] = None) -> str:
    """
    Calls the Flight Search Agent to find flight information.
    Passes the relevant query to the flight agent and returns its response.
    """
    print(f"\n[Orchestrator] Calling FlightSearchAgent with query: '{query}'")
    # Pass the orchestrator's callbacks down to the sub-agent via config
    # Ensure callbacks is a list, even if None is passed
    result = flight_agent_instance.run(query, callbacks=callbacks) # Pass callbacks to the run method
    return result # The run method already returns the output string

# Use the Pydantic model for args_schema
@tool("call_hotel_booking_agent", args_schema=HotelBookingOrchestratorInput)
def call_hotel_booking_agent(query: str, callbacks: Optional[List[BaseCallbackHandler]] = None) -> str:
    """
    Calls the Hotel Booking Agent to find hotel information.
    Passes the relevant query to the hotel agent and returns its response.
    """
    print(f"\n[Orchestrator] Calling HotelBookingAgent with query: '{query}'")
    # Pass the orchestrator's callbacks down to the sub-agent via config
    # Ensure callbacks is a list, even if None is passed
    result = hotel_agent_instance.run(query, callbacks=callbacks) # Pass callbacks to the run method
    return result # The run method already returns the output string

# The run methods in FlightSearchAgent and HotelBookingAgent are already correctly
# passing the 'callbacks' argument from their signature to the executor's invoke config.
# No changes needed there.

# --- 4. Build the Orchestrator Agent (same as before) ---
orchestrator_tools = [call_flight_search_agent, call_hotel_booking_agent]

orchestrator_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a highly capable travel planning orchestrator AI.
     Delegate to the appropriate specialized agent. Provide a clear, concise answer.
     """),
    ("user", "{input}"),
    ("placeholder", "{agent_scratchpad}"),
])

orchestrator_agent = create_tool_calling_agent(llm, orchestrator_tools, orchestrator_prompt)
# Orchestrator's verbose is now managed by callbacks
# Ensure the agent executor is created with the updated tools list
orchestrator_agent_executor = AgentExecutor(agent=orchestrator_agent, tools=orchestrator_tools, verbose=False)

# --- 5. Implement Custom Callback Handler for Observability ---

class MyAgentMonitor(BaseCallbackHandler):
    """A custom callback handler to monitor agent execution."""

    def __init__(self):
        self.llm_calls = 0
        self.tool_calls = 0
        self.start_time = {} # To track latency for specific runs
        self.end_time = {}
        self.token_usage = {} # {run_id: {'prompt_tokens': N, 'completion_tokens': M, 'total_tokens': K}}
        self.errors = []
        self.run_history = {} # Store a trace of messages/events for each run

    def on_chain_start(self, serialized: Dict[str, Any], run_id: UUID, parent_run_id: Optional[UUID] = None, **kwargs: Any) -> None:
        """Run when chain starts running."""
        chain_name = serialized.get("name", serialized.get("id", ["UnknownChain"])[-1])
        self.start_time[run_id] = time.time()
        self.run_history.setdefault(run_id, []).append(f"--- Chain '{chain_name}' (ID: {run_id}, Parent: {parent_run_id}) START ---")
        print(f"\n[MONITOR] Chain '{chain_name}' START (ID: {run_id}, Parent: {parent_run_id})")

    def on_chain_end(self, outputs: Dict[str, Any], run_id: UUID, parent_run_id: Optional[UUID] = None, **kwargs: Any) -> None:
        """Run when chain ends running."""
        chain_name = kwargs.get("name", "UnknownChain") # Getting chain name from kwargs here if available
        self.end_time[run_id] = time.time()
        duration = self.end_time[run_id] - self.start_time.get(run_id, self.end_time[run_id])
        self.run_history.setdefault(run_id, []).append(f"--- Chain '{chain_name}' END (ID: {run_id}, Duration: {duration:.2f}s) ---")
        print(f"[MONITOR] Chain '{chain_name}' END (ID: {run_id}, Duration: {duration:.2f}s)")
        print(f"[MONITOR] Final Output for {run_id}: {outputs.get('output', 'N/A')[:100]}...\n")


    def on_llm_start(self, serialized: Dict[str, Any], prompts: List[str], run_id: UUID, parent_run_id: Optional[UUID] = None, **kwargs: Any) -> None:
        """Run when LLM starts running."""
        self.llm_calls += 1
        # Be careful with large prompts; truncate or summarize
        prompt_preview = prompts[0][:100] + "..." if prompts and prompts[0] else "N/A"
        self.run_history.setdefault(run_id, []).append(f"  [LLM Start] Prompt: {prompt_preview}")
        print(f"  [MONITOR] LLM Call {self.llm_calls} (Run: {run_id}, Parent: {parent_run_id}): Start")

    def on_llm_end(self, response: LLMResult, run_id: UUID, parent_run_id: Optional[UUID] = None, **kwargs: Any) -> None:
        """Run when LLM ends running."""
        token_usage = response.llm_output.get("token_usage", {}) if response.llm_output else {}
        self.token_usage.setdefault(run_id, {'prompt_tokens': 0, 'completion_tokens': 0, 'total_tokens': 0})
        for key, value in token_usage.items():
            if key in self.token_usage[run_id]:
                self.token_usage[run_id][key] += value
            else:
                 self.token_usage[run_id][key] = value # Handle new token types if they appear

        self.run_history.setdefault(run_id, []).append(f"  [LLM End] Tokens (P:{token_usage.get('prompt_tokens',0)}, C:{token_usage.get('completion_tokens',0)})")
        print(f"  [MONITOR] LLM Call End (Run: {run_id}): Tokens: {token_usage.get('total_tokens', 'N/A')}")


    def on_tool_start(self, serialized: Dict[str, Any], input_str: str, run_id: UUID, parent_run_id: Optional[UUID] = None, **kwargs: Any) -> Any:
        """Run when tool starts running."""
        self.tool_calls += 1
        tool_name = serialized.get("name", "UnknownTool")
        self.run_history.setdefault(run_id, []).append(f"  [Tool Start] '{tool_name}' with input: {input_str[:100]}...")
        print(f"  [MONITOR] Tool Call {self.tool_calls} (Run: {run_id}, Parent: {parent_run_id}): '{tool_name}' Input: {input_str}")

    def on_tool_end(self, output: str, run_id: UUID, parent_run_id: Optional[UUID] = None, **kwargs: Any) -> None:
        """Run when tool ends running."""
        self.run_history.setdefault(run_id, []).append(f"  [Tool End] Output: {output[:100]}...")
        print(f"  [MONITOR] Tool Call End (Run: {run_id}). Output: {output[:50]}...")

    def on_tool_error(self, error: Exception, run_id: UUID, parent_run_id: Optional[UUID] = None, **kwargs: Any) -> None:
        """Run when tool errors."""
        tool_name = kwargs.get("name", "UnknownTool")
        self.errors.append({"run_id": run_id, "parent_run_id": parent_run_id, "error": str(error), "type": "tool", "tool_name": tool_name})
        self.run_history.setdefault(run_id, []).append(f"  [Tool Error] '{tool_name}': {error}")
        print(f"  [MONITOR] ERROR in Tool '{tool_name}' (Run: {run_id}, Parent: {parent_run_id}): {error}")

    def on_agent_action(self, action: Any, run_id: UUID, parent_run_id: Optional[UUID] = None, **kwargs: Any) -> Any:
        """Run on agent action."""
        self.run_history.setdefault(run_id, []).append(f"  [Agent Action] Tool: {action.tool}, Input: {action.tool_input}")
        # print(f"  [MONITOR] Agent Action (Run: {run_id}, Parent: {parent_run_id}): Tool='{action.tool}', Input='{action.tool_input}'")

    def on_agent_finish(self, finish: Any, run_id: UUID, parent_run_id: Optional[UUID] = None, **kwargs: Any) -> None:
        """Run on agent end."""
        self.run_history.setdefault(run_id, []).append(f"  [Agent Finish] Output: {finish.return_values['output'][:100]}...")
        # print(f"  [MONITOR] Agent Finish (Run: {run_id}, Parent: {parent_run_id}): Output='{finish.return_values['output']}'")

    # Added handler for text output, often useful for intermediate thoughts
    def on_text(self, text: str, run_id: Optional[UUID] = None, parent_run_id: Optional[UUID] = None, **kwargs: Any) -> None:
        """Run on text output."""
        # Text output might not always have a run_id if not part of a traced chain/tool
        # Handle cases where run_id or parent_run_id might be None
        key = run_id if run_id is not None else parent_run_id
        if key is not None:
             self.run_history.setdefault(key, []).append(f"  [Text Output] {text}")
        # print(f"[MONITOR] Text Output (Run: {run_id}, Parent: {parent_run_id}): {text}")


    def print_summary(self):
        print("\n--- MONITORING SUMMARY ---")
        print(f"Total LLM Calls: {self.llm_calls}")
        print(f"Total Tool Calls: {self.tool_calls}")
        total_total_tokens = sum(tu.get('total_tokens', 0) for tu in self.token_usage.values())
        print(f"Total Tokens Consumed (Overall Runs): {total_total_tokens}")
        print(f"Number of Errors: {len(self.errors)}")
        if self.errors:
            print("Errors detected:")
            for error_info in self.errors:
                print(f"  - Run ID: {error_info['run_id']}, Parent ID: {error_info['parent_run_id']}, Type: {error_info['type']}, Tool: {error_info.get('tool_name', 'N/A')}, Error: {error_info['error']}")

        print("\n--- DETAILED RUN HISTORY ---")
        for run_id, history in self.run_history.items():
            print(f"\n***** Run ID: {run_id} *****")
            for item in history:
                print(item)
            duration = self.end_time.get(run_id, self.start_time.get(run_id, 0)) - self.start_time.get(run_id, 0)
            tokens = self.token_usage.get(run_id, {'total_tokens': 'N/A'})
            print(f"  Run Duration: {duration:.2f}s, Tokens: {tokens.get('total_tokens', 'N/A')}")
            print(f"****************************")


# --- 6. Run Agent with Callbacks ---
my_monitor = MyAgentMonitor()

print("--- Running Orchestrator Agent with Custom Monitor ---")

# Pass the custom callback handler to the invoke method of the orchestrator
print("\nUser: Find flights from Montreal to Paris on August 15, 2025.")
orchestrator_agent_executor.invoke(
    {"input": "Find flights from Montreal to Paris on August 15, 2025."},
    config={"callbacks": [my_monitor]}
)

print("\nUser: I need a hotel in Paris for August 15 to August 20, 2025 for 2 guests.")
orchestrator_agent_executor.invoke(
    {"input": "I need a hotel in Paris for August 15 to August 20, 2025 for 2 guests."},
    config={"callbacks": [my_monitor]}
)

# Run a query that might involve both (orchestrator decides)
print("\nUser: Plan a trip: flights from Montreal to Paris on Aug 15, 2025, AND a hotel in Paris from Aug 15 to Aug 20, 2025 for 2 people.")
orchestrator_agent_executor.invoke(
    {"input": "Plan a trip: flights from Montreal to Paris on Aug 15, 2025, AND a hotel in Paris from Aug 15 to Aug 20, 2025 for 2 people."},
    config={"callbacks": [my_monitor]}
)


# --- 7. Print Monitoring Summary ---
my_monitor.print_summary()

--- Running Orchestrator Agent with Custom Monitor ---

User: Find flights from Montreal to Paris on August 15, 2025.
[MONITOR] Chain 'UnknownChain' END (ID: 8938f5cb-9935-4258-9e7d-07d22b713dac, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: b81a3b2c-724d-4721-86da-06dcdb7e0fb7, Duration: 0.00s)
[MONITOR] Final Output for b81a3b2c-724d-4721-86da-06dcdb7e0fb7: N/A...

[MONITOR] Chain 'UnknownChain' END (ID: d9d0caf5-2ffe-47db-947a-79768b31d040, Duration: 0.00s)
[MONITOR] Final Output for d9d0caf5-2ffe-47db-947a-79768b31d040: N/A...

[MONITOR] Chain 'UnknownChain' END (ID: 48696dab-7bef-4a14-a574-58752ce88efe, Duration: 0.00s)
  [MONITOR] LLM Call 1 (Run: 13e0a726-8941-4946-8e34-6efd513b6770, Parent: 991a5252-ec3f-4f69-b51b-c4910ea6f0dd): Start


  [MONITOR] LLM Call End (Run: 13e0a726-8941-4946-8e34-6efd513b6770): Tokens: N/A
[MONITOR] Chain 'UnknownChain' END (ID: 1e64ca70-87d1-40f4-84be-e09a4bd7e52a, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: 991a5252-ec3f-4f69-b51b-c4910ea6f0dd, Duration: 0.00s)
  [MONITOR] Tool Call 1 (Run: 7b6b602e-5db7-4efe-8ea8-99b84de35972, Parent: 37710cef-17c1-461b-b79c-46f6f03face5): 'call_flight_search_agent' Input: {'query': 'flights from Montreal to Paris on August 15, 2025'}

[Orchestrator] Calling FlightSearchAgent with query: 'flights from Montreal to Paris on August 15, 2025'
[MONITOR] Chain 'UnknownChain' END (ID: e4f8ae4f-e323-4c0e-b721-84f6d6e36713, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: 0ccf0d57-b24d-4420-a167-3da3dcd48d60, Duration: 0.00s)
[MONITOR] Final Output for 0ccf0d57-b24d-4420-a167-3da3dcd48d60: N/A...

[MONITOR] Chain 'UnknownChain' END (ID: a80590b3-7338-467e-b919-5048082b04f1, Duration: 0.00s)
[MONITOR] Final Output for a80590b3-7338-467e-b919-5

  [MONITOR] LLM Call End (Run: 03b9cbea-2889-4242-ab51-dfee8f476f1a): Tokens: N/A
[MONITOR] Chain 'UnknownChain' END (ID: ca75cb7a-c9e3-4dc7-ae91-2d4792dcb9d1, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: 066af13d-a7b6-4c3c-a2b2-5dd7cc2991e5, Duration: 0.00s)
  [MONITOR] Tool Call 2 (Run: f5fd83aa-2346-4938-81fe-f35b01908764, Parent: 4e994666-6945-4077-8b51-8ea0141d2e9c): 'search_flights_api' Input: {'origin': 'Montreal', 'destination': 'Paris', 'date': '2025-08-15'}
  [FlightSearchAgent] Simulating Flight API call for: Montreal -> Paris on 2025-08-15


  [MONITOR] Tool Call End (Run: f5fd83aa-2346-4938-81fe-f35b01908764). Output: [{'flight_id': 'AC123', 'price': 750}, {'flight_id': 'AF456', 'price': 820}]...
[MONITOR] Chain 'UnknownChain' END (ID: ff4e61c2-c458-4d6a-803a-e7f0ad35bfca, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: 13db800b-f9df-447f-9d24-9b879ba2f534, Duration: 0.00s)
[MONITOR] Final Output for 13db800b-f9df-447f-9d24-9b879ba2f534: N/A...

[MONITOR] Chain 'UnknownChain' END (ID: 887d3889-3fe6-4638-9e1b-95fff840006a, Duration: 0.00s)
[MONITOR] Final Output for 887d3889-3fe6-4638-9e1b-95fff840006a: N/A...

[MONITOR] Chain 'UnknownChain' END (ID: fd1f5398-1665-4749-a217-3a673211d7a6, Duration: 0.00s)
  [MONITOR] LLM Call 3 (Run: 0e19f522-d660-4b70-b196-8a31fe6de83e, Parent: 90f37a98-4934-40a9-897c-f7ac981e9f6b): Start


  [MONITOR] LLM Call End (Run: 0e19f522-d660-4b70-b196-8a31fe6de83e): Tokens: N/A
[MONITOR] Chain 'UnknownChain' END (ID: 797b9a0b-dfb0-4f40-80b5-3210a22f0b3b, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: 90f37a98-4934-40a9-897c-f7ac981e9f6b, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: 4e994666-6945-4077-8b51-8ea0141d2e9c, Duration: 0.00s)
[MONITOR] Final Output for 4e994666-6945-4077-8b51-8ea0141d2e9c: Here are some flight options from Montreal to Paris on August 15, 2025:

1. **Flight AC123** - Price...

  [MONITOR] Tool Call End (Run: 7b6b602e-5db7-4efe-8ea8-99b84de35972). Output: Here are some flight options from Montreal to Pari...
[MONITOR] Chain 'UnknownChain' END (ID: eb79a8b3-8e0a-45dd-9166-078df63fffe7, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: a6b09da3-ad9d-49da-824b-4f799e61e58a, Duration: 0.00s)
[MONITOR] Final Output for a6b09da3-ad9d-49da-824b-4f799e61e58a: N/A...

[MONITOR] Chain 'UnknownChain' END (ID: 31dad814-bdc2-4f1a-a4df-354

...

  [MONITOR] LLM Call End (Run: 2a716089-86aa-43e2-b1d5-7ebc2a3e950c): Tokens: N/A
[MONITOR] Chain 'UnknownChain' END (ID: c484f741-d1b0-495f-80d3-7788e32b5dfe, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: d769c27e-4e2f-4433-8c4a-a4a2cecfca9f, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: 28b0ec8e-5bb0-4727-acf6-9a3f07f98c68, Duration: 0.00s)
[MONITOR] Final Output for 28b0ec8e-5bb0-4727-acf6-9a3f07f98c68: Here are some flight options from Montreal to Paris on August 15, 2025:

1. **Flight AC123** - Price...

  [MONITOR] Tool Call End (Run: 86ffcd5f-e16e-4bf2-b31f-65547e7e2913). Output: Here are some flight options from Montreal to Pari...
  [MONITOR] Tool Call 7 (Run: 1969db5f-12b9-4514-ab5d-de4dcc0c5bc3, Parent: 46b6e706-50c9-40fd-8fce-b512ac534053): 'call_hotel_booking_agent' Input: {'query': 'hotel in Paris from Aug 15 to Aug 20, 2025 for 2 people'}

[Orchestrator] Calling HotelBookingAgent with query: 'hotel in Paris from Aug 15 to Aug 20, 2025 for 2 people'


  [MONITOR] LLM Call End (Run: f6aac33e-238a-464b-9e83-0889ccf4bfe5): Tokens: N/A
[MONITOR] Chain 'UnknownChain' END (ID: 33677f03-87ac-472b-bdf0-65fed4ad7d09, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: 40aea2a6-3092-4326-95fb-aa8bda04c75f, Duration: 0.00s)
  [MONITOR] Tool Call 8 (Run: e42b266b-358e-480d-855f-552be24b6ac5, Parent: 842f5a6b-563e-4855-81b9-0b83168d0e9e): 'Google_Hotels_api' Input: {'location': 'Paris', 'check_in_date': '2025-08-15', 'check_out_date': '2025-08-20', 'num_guests': 2}
  [HotelBookingAgent] Simulating Hotel API call for: Paris for 2 guests


  [MONITOR] Tool Call End (Run: e42b266b-358e-480d-855f-552be24b6ac5). Output: [{'hotel_id': 'HP789', 'price_per_night': 200}, {'hotel_id': 'RS012', 'price_per_night': 150}]...
[MONITOR] Chain 'UnknownChain' END (ID: e9adf47e-89a0-4626-8482-66d79a59a377, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: 44f8011c-71f7-4fb0-931c-94ce43fee2c5, Duration: 0.00s)
[MONITOR] Final Output for 44f8011c-71f7-4fb0-931c-94ce43fee2c5: N/A...

[MONITOR] Chain 'UnknownChain' END (ID: 72b0b400-97af-42ba-9559-8506208ca70b, Duration: 0.00s)
[MONITOR] Final Output for 72b0b400-97af-42ba-9559-8506208ca70b: N/A...

[MONITOR] Chain 'UnknownChain' END (ID: 5a061a7e-92bb-4a9b-861f-b428467d541a, Duration: 0.00s)
  [MONITOR] LLM Call 13 (Run: 0476464b-2761-4e98-9fab-ef96332eb156, Parent: 383a5b7e-5e18-4b3f-a1f1-78bffe5c52b2): Start


  [MONITOR] LLM Call End (Run: 0476464b-2761-4e98-9fab-ef96332eb156): Tokens: N/A
[MONITOR] Chain 'UnknownChain' END (ID: 38f8ecac-7dc4-4448-8fcb-9e1018e1b267, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: 383a5b7e-5e18-4b3f-a1f1-78bffe5c52b2, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: 842f5a6b-563e-4855-81b9-0b83168d0e9e, Duration: 0.00s)
[MONITOR] Final Output for 842f5a6b-563e-4855-81b9-0b83168d0e9e: Here are some hotel options in Paris for your stay from August 15 to August 20, 2025, for 2 people:
...

  [MONITOR] Tool Call End (Run: 1969db5f-12b9-4514-ab5d-de4dcc0c5bc3). Output: Here are some hotel options in Paris for your stay...
[MONITOR] Chain 'UnknownChain' END (ID: 4ab2eee6-d669-4608-8856-e165b6f018d0, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: baa92869-8c31-4cbe-a7a2-e427a3b1a7c4, Duration: 0.00s)
[MONITOR] Final Output for baa92869-8c31-4cbe-a7a2-e427a3b1a7c4: N/A...

[MONITOR] Chain 'UnknownChain' END (ID: 1f726451-76c7-45a2-acbd-06d

  [MONITOR] LLM Call End (Run: f65870ff-955e-411a-af01-d2add208a6e6): Tokens: N/A
[MONITOR] Chain 'UnknownChain' END (ID: 4d5759e0-c3b2-48cb-bba3-32a956b4e0ac, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: dac80273-de4a-4487-b6aa-34c588eddc3e, Duration: 0.00s)
[MONITOR] Chain 'UnknownChain' END (ID: 46b6e706-50c9-40fd-8fce-b512ac534053, Duration: 0.00s)
[MONITOR] Final Output for 46b6e706-50c9-40fd-8fce-b512ac534053: Here are some options for your trip from Montreal to Paris:

### Flights from Montreal to Paris on A...


--- MONITORING SUMMARY ---
Total LLM Calls: 14
Total Tool Calls: 8
Total Tokens Consumed (Overall Runs): 0
Number of Errors: 0

--- DETAILED RUN HISTORY ---

***** Run ID: 8938f5cb-9935-4258-9e7d-07d22b713dac *****
--- Chain 'UnknownChain' END (ID: 8938f5cb-9935-4258-9e7d-07d22b713dac, Duration: 0.00s) ---
  Run Duration: 1748390016.63s, Tokens: N/A
****************************

***** Run ID: b81a3b2c-724d-4721-86da-06dcdb7e0fb7 *****
--- Chain 'UnknownChain

Explanation and Key Learnings:

1. BaseCallbackHandler: This is the core class in LangChain for creating custom monitoring. You inherit from it and override specific on_... methods.

2. Event Hooks:

* * on_chain_start/on_chain_end: Triggered when any LangChain Runnable (including agents) starts or finishes. Great for tracking overall run duration.

* * on_llm_start/on_llm_end: Triggered for every LLM call. on_llm_end provides response.llm_output.token_usage which contains prompt_tokens, completion_tokens, and total_tokens.
on_tool_start/on_tool_end/

* * on_tool_error: Triggered when a tool is invoked. Useful for tracking tool usage, success/failure, and individual tool latency.

* * on_agent_action/on_agent_finish: Specific to agents, showing the agent's chosen action and its final output.

3. Passing Callbacks Down the Chain: Notice how callbacks is passed from the orchestrator's invoke call down to the run methods of FlightSearchAgent and HotelBookingAgent. This ensures that a single MyAgentMonitor instance tracks events across the entire nested agent hierarchy.

4. run_id and parent_run_id: These UUIDs are crucial for tracing. Each event (LLM call, tool call, chain execution) has a unique run_id. parent_run_id links it to the larger execution trace, allowing you to reconstruct the full flow (e.g., an LLM call within a tool call within an agent chain).

5. Metrics Captured:

* * Total Calls: llm_calls, tool_calls.

* * Latency: Calculated using start_time and end_time for chains.

* * Token Usage: Aggregated from on_llm_end events.

* * Errors: Captured via on_tool_error (you could also add on_llm_error, on_chain_error).

* * Execution Trace: run_history provides a sequential log of events within each run.

# chapter 6: Multi-Agent Applications

To build a truly intelligent and robust flight planning AI agent, we move towards distributed, multi-tasking agents that collaborate. This architecture allows you to break down complex problems into smaller, manageable tasks handled by specialized agents, leading to more scalable, maintainable, and powerful AI applications.

Distributed, Multi-tasking Agents for Complex Actions

1. Introduction to Tools, Agents, and Autonomous Behavior:

* * Tools: These are functions, APIs, or external systems that an LLM agent can use to interact with the real world or access information beyond its training data (e.g., our search_flights_api, get_airport_info tools).

* * Agents: An agent is an LLM enhanced with tools and a reasoning engine (often a prompt that guides its thought process). It can autonomously decide when and how to use its tools to achieve a goal. This autonomous behavior is what allows it to go beyond single-turn responses to multi-step problem-solving.

* * Autonomous Behavior: The ability of an agent to plan, execute, and adapt its actions without constant human intervention, driven by its internal reasoning and external observations.


2. Tools for Building Multi-Agent Systems:

* * LangChain Agents: As we've seen, LangChain provides a robust framework (AgentExecutor, create_tool_calling_agent, tools) to define agents, equip them with capabilities, and manage their execution loops.

* * LangChain Toolkits: LangChain offers pre-built toolkits (e.g., for Wikipedia, Google Search, various APIs) that you can easily integrate into your agents.

* * Other Frameworks: Beyond LangChain, frameworks like CrewAI and AutoGen (Microsoft) are specifically designed to facilitate multi-agent collaboration, offering more advanced patterns for task delegation, communication, and human-in-the-loop interactions. These often abstract away some of the communication complexities.


3. Designing Task-Specific Agents:

In a multi-agent system, each agent typically has a specialized role, much like departments in a company:

* * Planner Agent:

* * * Role: Takes a high-level user request and breaks it down into a sequence of smaller, actionable sub-tasks.

* * * Output: A structured list of tasks for the Executor.

* * * Flight Planning Example: User asks "Plan a summer trip to Europe." Planner might output: "1. Find flights to Paris. 2. Find hotels in Paris. 3. Find flights from Paris to Rome. 4. Find hotels in Rome. 5. Summarize itinerary."

* * Executor Agent:

Role: Receives a specific task from the Planner and executes it using its own set of specialized tools. It might delegate to other sub-agents.

Output: The results of the executed task (e.g., flight options, hotel details).

Flight Planning Example: Receives "Find flights to Paris," then calls the FlightSearchAgent's API (our call_flight_search_agent tool).

* * Summarizer/Reporter Agent:

Role: Takes raw results from the Executor(s) and synthesizes them into a coherent, user-friendly summary or report.

Output: The final answer presented to the user.

Flight Planning Example: Combines flight and hotel details into a readable itinerary with dates, times, and prices.

* * Evaluator/Critic Agent (Optional but powerful):

* * * Role: Reviews the output or actions of other agents for correctness, completeness, or adherence to constraints.

* * * Flight Planning Example: Checks if all segments of a multi-city trip are covered, or if the proposed flights fit the user's budget.


4. Communication Protocols Between Agents:

The choice of communication protocol depends on the complexity, scale, and deployment environment:

* Direct Function Calls (Local Simulation): As in our "Hands-on" example below, Python functions can directly call each other. Simple for local testing.

* HTTP/REST APIs (Microservices): Each agent exposes its capabilities via a RESTful API. Agents communicate by making HTTP requests. This is common for distributed deployments.

* Message Queues (Asynchronous Communication): Agents publish tasks/results to a central message queue (e.g., RabbitMQ, Kafka, AWS SQS). Other agents subscribe to relevant topics. This enables decoupled, asynchronous, and scalable communication.

* Shared Databases/Knowledge Bases: Agents can write and read structured information to/from a shared database or vector store.

* Agent2Agent (A2A) Protocol: As discussed, this emerging standard aims to provide a universal, structured, and secure way for agents to communicate, abstracting away underlying transport mechanisms.

Hands-on: Create a Multi-Agent System for a Business Use Case (Trip Planning)
We will build a simple multi-agent system for trip planning, comprising:

1. Planner Agent: Determines the sub-tasks for a trip.

2. Executor Agent: Calls specialized tools (our FlightSearchAgent and HotelBookingAgent from previous examples) to perform the tasks.

3. Summarizer Agent: Compiles the results into a final itinerary.

Note: For simplicity, we'll reuse the FlightSearchAgent and HotelBookingAgent classes directly as the "specialized tools" called by the Executor.

In [ ]:
import os
import time
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from typing import List, Dict, Any, Optional
import json # Import json module for parsing
from pydantic import BaseModel, Field # Import BaseModel and Field
from langchain_core.output_parsers import StrOutputParser # Import StrOutputParser

# Initialize LLM (same as previous examples)
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# --- Re-define Specialized Agents from previous sections (Flight and Hotel) ---
# These act as "external services" that our Executor Agent will call.
# Their 'run' methods simulate an API endpoint.

class FlightSearchAgent:
    def __init__(self, llm_instance):
        self.llm = llm_instance
        @tool("search_flights_api", args_schema={"origin": str, "destination": str, "date": str})
        def _search_flights_tool(origin: str, destination: str, date: str) -> List[dict]:
            """Searches for available flights based on origin, destination, and date.""" # Added docstring
            print(f"  [FlightSearchAgent] Simulating Flight API call for: {origin} -> {destination} on {date}")
            time.sleep(0.5)
            if "Montreal" in origin and "Paris" in destination and "2025-08-15" in date:
                return [{"flight_id": "AC123", "price": 750, "airline": "Air Canada"}, {"flight_id": "AF456", "price": 820, "airline": "Air France"}]
            elif "Paris" in origin and "Rome" in destination and "2025-08-20" in date:
                return [{"flight_id": "AZ789", "price": 120, "airline": "Alitalia"}, {"flight_id": "AF101", "price": 150, "airline": "Air France"}]
            return []
        self.tools = [_search_flights_tool]
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful flight search assistant. Use the 'search_flights_api' tool."),
            ("user", "{input}"), ("placeholder", "{agent_scratchpad}")])
        self.agent = create_tool_calling_agent(self.llm, self.tools, self.prompt)
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, verbose=False) # Internal verbose can be off

    def run(self, query: str) -> str:
        result = self.executor.invoke({"input": query})
        # The agent executor returns a dictionary, the output is under the 'output' key
        return result['output']

class HotelBookingAgent:
    def __init__(self, llm_instance):
        self.llm = llm_instance
        @tool("Google Hotels_api", args_schema={"location": str, "check_in_date": str, "check_out_date": str, "num_guests": int})
        def _Google_Hotels_tool(location: str, check_in_date: str, check_out_date: str, num_guests: int) -> List[dict]:
            """Searches for available hotels based on location, dates, and number of guests.""" # Added docstring
            print(f"  [HotelBookingAgent] Simulating Hotel API call for: {location} for {num_guests} guests")
            time.sleep(0.3)
            if "Paris" in location and "2025-08-15" in check_in_date:
                return [{"hotel_id": "HP789", "name": "Hotel Paradis", "price_per_night": 200, "stars": 4}, {"hotel_id": "RS012", "name": "Riverside Suites", "price_per_night": 150, "stars": 3}]
            elif "Rome" in location and "2025-08-20" in check_in_date:
                return [{"hotel_id": "GHZXC", "name": "Grand Hotel Roma", "price_per_night": 180, "stars": 4}]
            return []
        self.tools = [_Google_Hotels_tool]
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful hotel search assistant. Use the 'Google Hotels_api' tool."),
            ("user", "{input}"), ("placeholder", "{agent_scratchpad}")])
        self.agent = create_tool_calling_agent(self.llm, self.tools, self.prompt)
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, verbose=False)

    def run(self, query: str) -> str:
        result = self.executor.invoke({"input": query})
        # The agent executor returns a dictionary, the output is under the 'output' key
        return result['output']

# Instantiate our specialized "service" agents
flight_service = FlightSearchAgent(llm)
hotel_service = HotelBookingAgent(llm)


# --- 1. Planner Agent ---
# Role: Breaks down a complex request into a list of structured sub-tasks.
class Task(BaseModel):
    task: str = Field(description="A concise description of the sub-task.")
    type: str = Field(description="The type of task, e.g., 'flight_search', 'hotel_search', 'summarize'.")
    details: Dict[str, Any] = Field(description="Key-value pairs of details for the task, e.g., {'origin': '...', 'destination': '...'}.")

@tool("break_down_request", args_schema={"request": str})
def break_down_request(request: str) -> List[Dict[str, Any]]:
    """
    Breaks down a complex user request into a list of structured tasks.
    Each task should have a 'task' description, a 'type' ('flight_search', 'hotel_search', 'summarize'),
    and 'details' (e.g., origin, destination, date, location, check_in_date, num_guests).
    Always include a final 'summarize' task to compile results.
    """
    # Modified system prompt to avoid potential parsing issues with the example JSON structure
    planner_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a task planner for a travel agent.
         Your job is to break down a user's travel request into a sequence of discrete tasks.
         The output must be a JSON list of tasks. Each task object must have the following keys:
         - 'task' (string): A concise description of the sub-task.
         - 'type' (string): The type of task. Must be one of 'flight_search', 'hotel_search', or 'summarize'.
         - 'details' (dictionary): Key-value pairs of parameters for the task.
           - For 'flight_search': Include 'origin' (string), 'destination' (string), 'date' (string, YYYY-MM-DD format).
           - For 'hotel_search': Include 'location' (string), 'check_in_date' (string, YYYY-MM-DD format), 'check_out_date' (string, YYYY-MM-DD format), 'num_guests' (integer).
         Always include a final task of type 'summarize' at the end of the list to compile all findings.
         """),
        ("human", "User Request: {request}"),
    ])


    prompt_value = planner_prompt.invoke({"request": request})
    llm_output = llm.invoke(prompt_value) # llm.invoke expects a PromptValue or string
    llm_output_str = str(llm_output.content) # Get the string content

    tasks_raw = []
    try:
        # The LLM is instructed to output a JSON list directly
        tasks_raw = json.loads(llm_output_str)
    except json.JSONDecodeError as e:
        print(f"[Planner] Error parsing JSON from LLM: {e}")
        print(f"[Planner] Raw LLM Output: {llm_output_str}")
        # Return an empty list or a task indicating failure
        return [{"task": "Failed to parse planning tasks", "type": "summarize", "details": {"error": f"JSON parse error in planner output: {e}", "raw_output": llm_output_str}}]


    # Validate and convert to Pydantic objects if desired, or just use dicts
    # For robust parsing, you could use PydanticOutputParser with a list of Task models
    validated_tasks = []
    for t in tasks_raw:
        try:
            # Ensure all required fields are present and correctly named
            task_data = {
                "task": t.get("task", ""), # Provide default empty string if missing
                "type": t.get("type", ""),
                "details": t.get("details", {}) # Provide default empty dict if missing
            }
            # Attempt to create a Pydantic model instance for validation
            validated_task_obj = Task(**task_data)
            validated_tasks.append(validated_task_obj.dict())
        except Exception as e:
            print(f"[Planner] Warning: Could not validate task data: {t}. Error: {e}")
            # Optionally skip or log malformed tasks
            pass # Or raise the exception if strict validation is needed

    # Ensure a summarize task is always the last one, add if missing
    if not validated_tasks or validated_tasks[-1].get('type') != 'summarize':
         # Check if there's already a summary task, if so remove it before adding
         validated_tasks = [t for t in validated_tasks if t.get('type') != 'summarize']
         validated_tasks.append({"task": "Compile the final itinerary", "type": "summarize", "details": {}})


    return validated_tasks


# --- 2. Executor Agent ---
# Role: Executes tasks by calling the appropriate specialized agents/tools.
@tool("execute_flight_search", args_schema={"query": str})
def execute_flight_search(query: str) -> str:
    """Executes a flight search query by calling the Flight Search Agent."""
    print(f"\n[Executor] Delegating to Flight Search Agent with query: '{query}'")
    return flight_service.run(query)

@tool("execute_hotel_search", args_schema={"query": str})
def execute_hotel_search(query: str) -> str:
    """Executes a hotel search query by calling the Hotel Booking Agent."""
    print(f"\n[Executor] Delegating to Hotel Booking Agent with query: '{query}'")
    return hotel_service.run(query)

executor_tools = [execute_flight_search, execute_hotel_search]

executor_agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a task executor for a travel agent.
     You receive specific tasks from the Planner and execute them using your tools.
     If the task type is 'flight_search', use 'execute_flight_search'.
     If the task type is 'hotel_search', use 'execute_hotel_search'.
     Return the exact result from the tool call.
     """),
    # The input to the executor agent is a dictionary {"task_description": ..., "task_details": ...}
    # We need to format this into a string for the agent's input.
    ("user", "Execute the following task:\nTask Description: {task_description}\nTask Details: {task_details}"),
    ("placeholder", "{agent_scratchpad}"),
])

executor_agent = create_tool_calling_agent(llm, executor_tools, executor_agent_prompt)
executor_agent_executor = AgentExecutor(agent=executor_agent, tools=executor_tools, verbose=True) # Executor verbose ON to see its internal actions

# --- 3. Summarizer Agent ---
# Role: Takes all collected information and creates a final, human-readable itinerary.
@tool("summarize_itinerary", args_schema={"results_data": Dict[str, Any], "original_request": str})
def summarize_itinerary(results_data: Dict[str, Any], original_request: str) -> str:
    """
    Summarizes the flight and hotel search results into a concise travel itinerary.
    Input `results_data` should be a dictionary containing keys like 'flights' and 'hotels'.
    """
    summarizer_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a travel itinerary summarizer.
         You receive raw search results and an original user request.
         Compile all the information into a clear, attractive, and concise travel itinerary.
         Highlight key details like dates, locations, flight numbers, airlines, and hotel names/prices.
         If any information is missing or could not be found, clearly state it.
         """),
        ("human", "Original Request: {original_request}\n\nSearch Results:\n{results_data}"),
    ])
    summarizer_chain = summarizer_prompt | llm | StrOutputParser()
    summary = summarizer_chain.invoke({"original_request": original_request, "results_data": results_data})
    return summary


# --- Orchestrator/Main Workflow ---
# This is the main script that orchestrates the flow between the agents.
def orchestrate_travel_planning(user_request: str) -> str:
    print(f"\n***** Orchestrator: Starting to process request: '{user_request}' *****")

    # Step 1: Planner Agent breaks down the request
    print("\n[Orchestrator] Calling Planner Agent to break down the request...")
    # The invoke method expects a dictionary matching the prompt template variables.
    # The planner_prompt expects 'request'.
    # Call the tool's invoke method directly, which internally calls the function break_down_request
    tasks = break_down_request.invoke({"request": user_request})
    print(f"[Orchestrator] Planner identified tasks: {tasks}")

    collected_results = {"flights": [], "hotels": []}

    # Step 2: Executor Agent executes each task
    for task in tasks:
        task_type = task.get('type') # Use .get for safer access
        task_description = task.get('task', 'No description') # Use .get with default
        task_details = task.get('details', {}) # Use .get with default

        if task_type == 'flight_search':
            # Construct a query string for the Flight Search Agent
            # The executor agent receives {"task_description": ..., "task_details": ...}
            # and should use the task_details to formulate the tool call.
            # We pass the task details as a string, the executor agent's prompt
            # instructs it to use task_details to formulate the tool call.
            print(f"[Orchestrator] Executing Flight Search Task: {task_description} with details {task_details}")
            # Ensure the input dictionary matches the executor_agent_prompt variables
            result_str = executor_agent_executor.invoke({
                "task_description": task_description,
                "task_details": str(task_details) # Pass details as a string representation
            })['output']
            try:
                # Assuming the executor returns a JSON string of flight results from the specialized agent
                # We need to load this string as JSON.
                # The specialized agent's run method already returns the list of dicts directly
                # The Executor agent's output will contain this list directly if it uses the tool correctly
                # Need to ensure the executor agent doesn't wrap it in extra text.
                # Let's refine the executor agent prompt to strictly return the tool output.
                flights = json.loads(result_str) # Attempt to parse in case the executor returns a string
                collected_results['flights'].extend(flights)
            except json.JSONDecodeError:
                print(f"[Orchestrator] Warning: Could not parse flight results from executor output: {result_str}")
                collected_results['flights'].append({"error": "Failed to parse flight results", "raw_output": result_str})
            except Exception as e:
                 print(f"[Orchestrator] Warning: An unexpected error occurred processing flight results: {e} Raw output: {result_str}")
                 collected_results['flights'].append({"error": f"Unexpected error processing flight results: {e}", "raw_output": result_str})


        elif task_type == 'hotel_search':
            # Construct a query string for the Hotel Booking Agent
            # The executor agent receives {"task_description": ..., "task_details": ...}
            # and should use the task_details to formulate the tool call.
            # We pass the task details as a string, the executor agent's prompt
            # instructs it to use task_details to formulate the tool call.
            print(f"[Orchestrator] Executing Hotel Search Task: {task_description} with details {task_details}")
             # Ensure the input dictionary matches the executor_agent_prompt variables
            result_str = executor_agent_executor.invoke({
                "task_description": task_description,
                "task_details": str(task_details) # Pass details as a string representation
            })['output']
            try:
                # Assuming the executor returns a JSON string of hotel results from the specialized agent
                # Need to ensure the executor agent doesn't wrap it in extra text.
                # Let's refine the executor agent prompt to strictly return the tool output.
                hotels = json.loads(result_str) # Attempt to parse in case the executor returns a string
                collected_results['hotels'].extend(hotels)
            except json.JSONDecodeError:
                print(f"[Orchestrator] Warning: Could not parse hotel results from executor output: {result_str}")
                collected_results['hotels'].append({"error": "Failed to parse hotel results", "raw_output": result_str})
            except Exception as e:
                print(f"[Orchestrator] Warning: An unexpected error occurred processing hotel results: {e} Raw output: {result_str}")
                collected_results['hotels'].append({"error": f"Unexpected error processing hotel results: {e}", "raw_output": result_str})

        elif task_type == 'summarize':
            # This task is handled by the Orchestrator itself after all data collection
            # No need to invoke the executor for this type.
            print("[Orchestrator] Skipping Summarize task in execution loop (handled later).")
            pass # Handled after the loop

        else:
            print(f"[Orchestrator] Unknown task type: {task_type}. Skipping task: {task_description}")


    # Step 3: Summarizer Agent compiles the results
    print("\n[Orchestrator] Calling Summarizer Agent to compile the itinerary...")
    # The summarize_itinerary tool expects {"results_data": ..., "original_request": ...}
    # Ensure collected_results and original_request are passed correctly
    final_itinerary = summarize_itinerary.invoke({
        "results_data": collected_results,
        "original_request": user_request
    })

    print("\n***** Orchestrator: Finished processing request. *****")
    return final_itinerary


# --- Hands-on Exercise: Create a multi-agent system for a business use case ---
# Scenario: A multi-city trip planning request.

user_request_1 = "Plan a trip to Europe for August 15-22, 2025. First, I need flights from Montreal to Paris. Then, a hotel in Paris. After that, flights from Paris to Rome. Finally, a hotel in Rome."
final_response_1 = orchestrate_travel_planning(user_request_1)
print("\n--- FINAL ITINERARY ---")
print(final_response_1)
print("-----------------------\n")

user_request_2 = "Find flights from Montreal to Paris on 2025-08-15 and a hotel in Paris for 2 guests from 2025-08-15 to 2025-08-18."
final_response_2 = orchestrate_travel_planning(user_request_2)
print("\n--- FINAL ITINERARY ---")
print(final_response_2)
print("-----------------------\n")


***** Orchestrator: Starting to process request: 'Plan a trip to Europe for August 15-22, 2025. First, I need flights from Montreal to Paris. Then, a hotel in Paris. After that, flights from Paris to Rome. Finally, a hotel in Rome.' *****

[Orchestrator] Calling Planner Agent to break down the request...
[Planner] Error parsing JSON from LLM: Expecting value: line 1 column 1 (char 0)
[Planner] Raw LLM Output: ```json
[
    {
        "task": "Search for flights from Montreal to Paris",
        "type": "flight_search",
        "details": {
            "origin": "Montreal",
            "destination": "Paris",
            "date": "2025-08-15"
        }
    },
    {
        "task": "Search for a hotel in Paris",
        "type": "hotel_search",
        "details": {
            "location": "Paris",
            "check_in_date": "2025-08-15",
            "check_out_date": "2025-08-18",
            "num_guests": 1
        }
    },
    {
        "task": "Search for flights from Paris to Rome",
 

# chapter 7: Agentic Workflow Fundamentals

Great! We've covered the foundational concepts of LLM agents, their internal workings, and how to monitor them. Now, let's unlock the true power of autonomous AI with LangGraph's orchestration engine. This is where we build structured, multi-tasking, and collaborative agents that can handle complex, dynamic workflows – perfectly suited for our advanced flight planning AI.

Dive into LangGraph’s Orchestration Engine

Graph-based Orchestration Models:
Traditional AgentExecutor from LangChain is excellent for linear "Thought-Action-Observation" loops. However, real-world problems often require:

* Non-linear flows: Branching based on conditions (e.g., "If flight found, then ask for hotel; else, suggest alternative dates").

* Loops/Cycles: Iterating (e.g., "Keep searching until a suitable flight is found" or "Refine answer until satisfactory").

* Complex Dependencies: Multiple sub-tasks running in parallel or sequentially with intricate data hand-offs.

* Stateful Transitions: The ability for the workflow to "remember" its current progress and make decisions based on accumulating information.

LangGraph solves this by adopting a state machine and graph-based approach. You define nodes (steps) and edges (transitions between steps), allowing for explicit control over the flow, including cycles and conditional routing.

A Practical Guide to Coordinated LLM Agents Using LangGraph:

* Nodes (functions or agents): Each node in a LangGraph represents a specific step in your workflow. A node can be:

A simple Python function (e.g., Notes, format_output).
An LLM call.
A tool invocation.
An entire LangChain AgentExecutor (allowing you to nest agents within a larger graph).

* Edges (data/control flow): These define how the workflow moves from one node to another.
Direct Edges: Unconditionally move from Node A to Node B.
Conditional Edges: The next node is determined by a function that evaluates the current state and returns the name of the next node (or END to stop).

* Cycles (iteration, self-correction): LangGraph natively supports cycles. This is how you implement looping behavior, allowing agents to iteratively refine an answer, gather more information, or retry failed operations.

* State: LangGraph manages a shared state object that is passed between nodes. Each node receives the current state, performs its operations, and returns an update to the state. This allows all nodes to access and contribute to the accumulated knowledge and progress of the workflow. The state is defined using a TypedDict or a simple dict that represents the data structure of your graph's context.

Add memory or context passing between agents:
In LangGraph, memory is intrinsically tied to the state. The state object itself serves as the short-term memory of the current execution trace. You can design your state to include a messages list for conversational memory, or specific keys for extracted entities and results. LangGraph also provides Checkpointers for persisting this state to a database, enabling long-running conversations or resuming workflows after interruptions.

Node-based Task Design:
Think of each node as a microservice or a single, atomic operation.

* Atomic: Each node should ideally perform one distinct logical unit of work.

* Input/Output: Clearly define what each node expects from the state and what it contributes back to the state.

* Separation of Concerns: Keep your planner, executor, summarizer, and specific tool interactions in separate nodes for clarity and maintainability.


Async vs Sync Execution in Agentic Flows:

LangGraph (and LangChain) supports both synchronous (.invoke()) and asynchronous (.ainvoke(), .stream(), .astream()) execution.

* Synchronous: Simpler to write for sequential tasks, blocking until each step completes.

* Asynchronous: Essential for high-throughput, non-blocking operations, especially when dealing with network calls (LLMs, external APIs) or parallel execution. LangGraph can internally manage async operations within nodes if you define your nodes as async def functions.

Conditional Routing and Stateful Transitions:

This is a cornerstone of LangGraph. You define a "router" function as an edge that inspects the current state and returns the name of the next node to execute.

Integrating memory into LangGraph Workflows:

* Conversation History: A common pattern is to have a messages key in your graph state, and use LangGraph's add_messages function (or a custom reducer) to append new messages to it.

* Structured Data: Store extracted entities, API results, or summaries directly in dedicated keys within your state.

* Checkpointers: For persistence, you compile the graph with a checkpointer (e.g., InMemorySaver for development, or a database-backed one for production). This allows you to resume conversations or long-running tasks.


Hands-on Exercises Using LangGraph
We will build a complete trip planning system using LangGraph. This system will:

1. Take a user request.
2. Use a Planner Node to break it into tasks.
3. Use an Executor Node that conditionally calls FlightSearchAgent or HotelBookingAgent.
4. Implement looping behavior to process multiple tasks.
5. Use a Summarizer Node to compile results.
6. Maintain state throughout the process.
7. Include an Error Handling Node for robustness.

We'll reuse our FlightSearchAgent and HotelBookingAgent from the previous section as internal components (called by the Executor).

Prerequisites:

In [ ]:
!pip install langchain langchain-openai langgraph -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.4 MB/s eta 0:00:00


In [ ]:
#!pip install langchain langchain-openai langgraph -q
import os
import json
import time
from typing import List, Dict, Any, TypedDict, Callable, Optional
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.tools import tool
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langgraph.graph import StateGraph, END, START
from langgraph.graph.graph import CompiledGraph
from langgraph.checkpoint.memory import InMemorySaver # For state persistence
from langchain_core.output_parsers import StrOutputParser # Import StrOutputParser

# --- Re-initialize LLM (as in previous steps) ---
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# --- Re-define Specialized Agents (Flight and Hotel) ---
# These are still regular LangChain Agents, which will be invoked as "tools"
# within our LangGraph nodes.

class FlightSearchAgent:
    def __init__(self, llm_instance):
        self.llm = llm_instance
        @tool("search_flights_api", args_schema={"origin": str, "destination": str, "date": str})
        def _search_flights_tool(origin: str, destination: str, date: str) -> List[dict]:
            """Searches for flights between origin and destination on a specific date.""" # Added docstring
            print(f"  [FlightSearchAgent] Simulating Flight API call for: {origin} -> {destination} on {date}")
            time.sleep(0.5)
            if "Montreal" in origin and "Paris" in destination and "2025-08-15" in date:
                return [{"flight_id": "AC123", "price": 750, "airline": "Air Canada"}, {"flight_id": "AF456", "price": 820, "airline": "Air France"}]
            elif "Paris" in origin and "Rome" in destination and "2025-08-20" in date:
                return [{"flight_id": "AZ789", "price": 120, "airline": "Alitalia"}, {"flight_id": "AF101", "price": 150, "airline": "Air France"}]
            return []
        self.tools = [_search_flights_tool]
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful flight search assistant. Use the 'search_flights_api' tool."),
            ("user", "{input}"), ("placeholder", "{agent_scratchpad}")])
        self.agent = create_tool_calling_agent(self.llm, self.tools, self.prompt)
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, verbose=False)

    def run(self, query: str) -> str:
        try:
            result = self.executor.invoke({"input": query})
            return result['output']
        except Exception as e:
            return f"ERROR: Flight Search Agent failed with {str(e)}"

class HotelBookingAgent:
    def __init__(self, llm_instance):
        self.llm = llm_instance
        @tool("Google Hotels_api", args_schema={"location": str, "check_in_date": str, "check_out_date": str, "num_guests": int})
        def _Google_Hotels_tool(location: str, check_in_date: str, check_out_date: str, num_guests: int) -> List[dict]:
            """Searches for hotels in a given location for specified dates and number of guests.""" # Added docstring
            print(f"  [HotelBookingAgent] Simulating Hotel API call for: {location} for {num_guests} guests")
            time.sleep(0.3)
            if "Paris" in location and "2025-08-15" in check_in_date:
                return [{"hotel_id": "HP789", "name": "Hotel Paradis", "price_per_night": 200, "stars": 4}, {"hotel_id": "RS012", "name": "Riverside Suites", "price_per_night": 150, "stars": 3}]
            elif "Rome" in location and "2025-08-20" in check_in_date:
                return [{"hotel_id": "GHZXC", "name": "Grand Hotel Roma", "price_per_night": 180, "stars": 4}]
            return []
        self.tools = [_Google_Hotels_tool]
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are a helpful hotel search assistant. Use the 'Google Hotels_api' tool."),
            ("user", "{input}"), ("placeholder", "{agent_scratchpad}")])
        self.agent = create_tool_calling_agent(self.llm, self.tools, self.prompt)
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, verbose=False)

    def run(self, query: str) -> str:
        try:
            result = self.executor.invoke({"input": query})
            return result['output']
        except Exception as e:
            return f"ERROR: Hotel Search Agent failed with {str(e)}"

# Instantiate our specialized "service" agents (these will be called by the Executor node)
flight_service = FlightSearchAgent(llm)
hotel_service = HotelBookingAgent(llm)

# --- Define the Graph State ---
# This TypedDict defines the structure of the data that will be passed between nodes.
class TravelGraphState(TypedDict):
    request: str # The original user request
    tasks: List[Dict[str, Any]] # List of tasks generated by the Planner
    current_task_index: int # Index of the task being processed
    flight_results: List[Dict[str, Any]] # Accumulated flight results
    hotel_results: List[Dict[str, Any]] # Accumulated hotel results
    error_message: Optional[str] # For error handling
    final_itinerary: Optional[str] # The final output

# --- Define the Nodes ---

# Node 1: Planner Node
def planner_node(state: TravelGraphState) -> TravelGraphState:
    """
    Takes the user request and breaks it down into a list of tasks.
    Updates the 'tasks' and 'current_task_index' in the state.
    """
    print("\n[Node: Planner] Breaking down the request...")
    # Escape curly braces that are part of the JSON format description
    # CORRECTED: Escaped curly braces in the system message example JSON
    planner_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a meticulous task planner for a travel agent.
         Your job is to break down a user's travel request into a precise sequence of discrete tasks.
         Each task must be a JSON object with:
         - 'task': (string) A concise description of the sub-task.
         - 'type': (string) The type of task, e.g., 'flight_search', 'hotel_search'.
         - 'details': (dict) Key-value pairs of parameters for the task.
           For 'flight_search': {{"origin"}}, {{"destination"}}, {{"date"}}.
           For 'hotel_search': {{"location"}}, {{"check_in_date"}}, {{"check_out_date"}}, {{"num_guests"}}.
         Ensure you extract all necessary details for each task.
         Return only a JSON list of task objects.
         Example: `[{{\"task\": \"Find flights from NYC to London\", \"type\": \"flight_search\", \"details\": {{\"origin\": \"NYC\", \"destination\": \"London\", \"date\": \"2025-07-01\"}}}}, {{\\"task\\": \\"Find hotel in London\\", \\"type\\": \\"hotel_search\\", \\"details\\": {\\"location\\": \\"London\\", \\"check_in_date\\": \\"2025-07-01\\", \\"check_out_date\\": \\"2025-07-05\\", \\"num_guests\\": 1}}}]`
         """),
        ("human", "User Request: {request}. Assume today's date is 2025-05-26 if relative dates are used."),
    ])
    try:
        # Ensure the output parser correctly handles potential non-JSON responses from the LLM
        # and provides a helpful error. We'll keep the lambda x: json.loads(x) for now,
        # but add a more specific error message around it.
        planner_chain = planner_prompt | llm | (lambda x: json.loads(x))
        tasks_raw = planner_chain.invoke({"request": state['request']})

        # Basic validation - check if the top level is a list
        if not isinstance(tasks_raw, list):
             raise ValueError(f"Planner did not return a JSON list. Raw output: {tasks_raw}")

        tasks = []
        for t in tasks_raw:
            # Basic validation - check if each item is a dict and has required keys
            # CORRECTED: Also check if 'details' is a dictionary
            if isinstance(t, dict) and all(k in t for k in ['task', 'type', 'details']) and isinstance(t.get('details'), dict): # Use .get for safer access
                tasks.append(t)
            else:
                # Catch malformed *items* in the list, provide better error
                raise ValueError(f"Invalid task format received from planner: {t}")


        print(f"[Node: Planner] Identified {len(tasks)} tasks.")
        # Ensure current_task_index is always returned on success
        return {"tasks": tasks, "current_task_index": 0}
    except json.JSONDecodeError as e:
        error_msg = f"Planner output was not valid JSON. Error: {e}. Raw output: {tasks_raw if 'tasks_raw' in locals() else 'N/A'}"
        print(f"[Node: Planner] Error planning tasks: {error_msg}")
        # Ensure state keys are present even on error for graph consistency
        return {"error_message": error_msg, "tasks": [], "current_task_index": 0}
    except Exception as e:
        error_msg = f"Error planning tasks: {e}"
        print(f"[Node: Planner] Error planning tasks: {error_msg}")
        # Ensure state keys are present even on error for graph consistency
        return {"error_message": error_msg, "tasks": [], "current_task_index": 0}


# Node 2: Executor Node
def executor_node(state: TravelGraphState) -> TravelGraphState:
    """
    Executes the current task using the appropriate specialized agent/tool.
    Updates 'flight_results' or 'hotel_results' in the state.
    Advances 'current_task_index'.
    """
    # Check for error message from previous node before proceeding
    if state.get("error_message"):
        print("[Node: Executor] Skipping execution due to previous error.")
        return state # Return current state, error_handler will be next via router

    # Basic check for expected keys
    if 'tasks' not in state or 'current_task_index' not in state:
         error = "Internal workflow error: missing task information in executor."
         print(f"[Node: Executor] {error}")
         return {"error_message": error} # This state update will be caught by the router

    print(f"\n[Node: Executor] Executing task index: {state['current_task_index']}")
    tasks = state['tasks']
    current_index = state['current_task_index']

    if current_index >= len(tasks):
        print("[Node: Executor] No more tasks to execute.")
        return state # Should not happen if conditional routing is correct

    task = tasks[current_index]
    # Use .get for safety when accessing task keys
    task_type = task.get('type')
    task_details = task.get('details', {}) # Default to empty dict if details are missing

    new_flight_results = state.get('flight_results', [])
    new_hotel_results = state.get('hotel_results', [])
    error = None # Initialize error to None for this node's execution

    try:
        if task_type == 'flight_search':
             # Use .get for safety when accessing task_details keys
            origin = task_details.get('origin')
            destination = task_details.get('destination')
            date = task_details.get('date')
            if not all([origin, destination, date]):
                 raise ValueError(f"Missing required details for flight search: {task_details}")
            query = f"Find flights from {origin} to {destination} on {date}."
            result_str = flight_service.run(query) # Call the specialized agent
            if result_str and "ERROR" in result_str: raise Exception(result_str)
            # Attempt to parse result, handle potential non-JSON
            try:
                flights = json.loads(result_str)
                if not isinstance(flights, list):
                     raise ValueError(f"Flight service did not return a list: {result_str}")
                new_flight_results.extend(flights)
                print(f"[Node: Executor] Found {len(flights)} flights.")
            except json.JSONDecodeError:
                 raise ValueError(f"Flight service returned non-JSON data: {result_str}")


        elif task_type == 'hotel_search':
            # Use .get for safety when accessing task_details keys
            location = task_details.get('location')
            check_in_date = task_details.get('check_in_date')
            check_out_date = task_details.get('check_out_date')
            num_guests = task_details.get('num_guests')
            if not all([location, check_in_date, check_out_date, num_guests]):
                 raise ValueError(f"Missing required details for hotel search: {task_details}")
            # Ensure num_guests is an integer if it came from a string
            try:
                num_guests = int(num_guests) if isinstance(num_guests, str) else num_guests
                if not isinstance(num_guests, int):
                     raise ValueError(f"Invalid num_guests format: {num_guests}")
            except ValueError:
                 raise ValueError(f"Could not convert num_guests to integer: {num_guests}")


            query = f"Find hotels in {location} from {check_in_date} to {check_out_date} for {num_guests} guests."
            result_str = hotel_service.run(query) # Call the specialized agent
            if result_str and "ERROR" in result_str: raise Exception(result_str)
            # Attempt to parse result, handle potential non-JSON
            try:
                hotels = json.loads(result_str)
                if not isinstance(hotels, list):
                     raise ValueError(f"Hotel service did not return a list: {result_str}")
                new_hotel_results.extend(hotels)
                print(f"[Node: Executor] Found {len(hotels)} hotels.")
            except json.JSONDecodeError:
                 raise ValueError(f"Hotel service returned non-JSON data: {result_str}")

        elif task_type is None:
             raise ValueError(f"Task dictionary missing 'type' key: {task}")
        else:
            error = f"Unsupported task type: {task_type}"
            print(f"[Node: Executor] {error}")
            # For unsupported tasks, maybe skip and move to next? Or treat as error?
            # Let's treat as error for now for robustness.

    except Exception as e:
        error = f"Error executing task '{task.get('task', 'Unknown Task')}': {e}"
        print(f"[Node: Executor] {error}")

    next_index = current_index + 1

    # Update state
    return {
        "flight_results": new_flight_results,
        "hotel_results": new_hotel_results,
        "current_task_index": next_index,
        # Preserve existing error_message if any, only overwrite if a *new* error occurred in *this* node
        "error_message": state.get("error_message") or error
    }


# Node 3: Summarizer Node
def summarizer_node(state: TravelGraphState) -> TravelGraphState:
    """
    Compiles all collected results into a final itinerary.
    Updates 'final_itinerary' in the state.
    """
    # Check for error message from previous nodes before proceeding
    if state.get("error_message"):
        print("[Node: Summarizer] Skipping summarization due to previous error.")
        return state # Return current state, error_handler will be next via router

    print("\n[Node: Summarizer] Compiling final itinerary...")
    summarizer_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a professional travel itinerary creator.
         You receive raw flight and hotel search results along with the original user request.
         Compile all the information into a clear, attractive, and concise travel itinerary.
         Highlight key details: dates, locations, flight numbers, airlines, departure/arrival times, hotel names, prices, and number of guests.
         If any information is missing or could not be found, clearly state it.
         Start with a friendly greeting and provide a well-structured summary.
         """),
        ("human", "Original Request: {original_request}\n\nSearch Results:\nFlights: {flights}\nHotes: {hotels}"),
    ])

    try:
        summary_chain = summarizer_prompt | llm | StrOutputParser()
        final_itinerary = summary_chain.invoke({
            "original_request": state.get('request', 'User request unavailable.'), # Use .get for safety
            "flights": state.get('flight_results', []),
            "hotels": state.get('hotel_results', [])
        })
        print("[Node: Summarizer] Itinerary compiled.")
        # Return final_itinerary and ensure error_message is cleared if summarization was successful
        return {"final_itinerary": final_itinerary, "error_message": None}
    except Exception as e:
        print(f"[Node: Summarizer] Error summarizing itinerary: {e}")
        # Ensure error_message is returned to state
        return {"error_message": f"Error summarizing: {e}"}

# Node 4: Error Handler Node (Optional but good for robustness)
def error_handler_node(state: TravelGraphState) -> TravelGraphState:
    """
    Handles errors that occurred in previous nodes.
    Formats a user-friendly error message for the final output.
    """
    print(f"\n[Node: Error Handler] Processing error: {state.get('error_message', 'An unknown error occurred.')}")
    error_summary = f"I encountered an issue during your request: {state.get('error_message', 'An unknown error occurred.')}. Please try again or rephrase your request."
    # Return the final itinerary as the error message
    return {"final_itinerary": error_summary, "error_message": state.get('error_message')}


# --- Define the Conditional Edges (Routers) ---

def should_continue_execution(state: TravelGraphState) -> str:
    """
    Decides whether to continue executing tasks or move to summarization/error.
    This router is used AFTER the executor node.
    """
    # Check for error message from the current node execution first
    if state.get("error_message"):
        print("[Router (after Executor)] Routing to Error Handler due to error.")
        return "error_handler"

    # Basic check for expected keys - shouldn't be missing if Planner ran successfully and
    # Executor returned the required keys, but defensive check is good.
    if 'tasks' not in state or 'current_task_index' not in state:
         print("[Router (after Executor)] Missing 'tasks' or 'current_task_index' in state after executor. Routing to error.")
         # Note: The executor node should ideally set the error_message if these were missing,
         # but we add a fallback here.
         # state["error_message"] = "Internal workflow error: missing task information after executor."
         return "error_handler"


    current_index = state['current_task_index']
    tasks = state['tasks']

    if current_index < len(tasks):
        # More tasks to execute, go back to the executor
        print(f"[Router (after Executor)] More tasks to execute. Routing to Executor (Task {current_index + 1}/{len(tasks)}).")
        return "executor"
    else:
        # All tasks processed, go to the summarizer
        print("[Router (after Executor)] All tasks processed. Routing to Summarizer.")
        return "summarizer"


# --- Build the LangGraph Workflow ---

builder = StateGraph(TravelGraphState)

# Add nodes
builder.add_node("planner", planner_node)
builder.add_node("executor", executor_node)
builder.add_node("summarizer", summarizer_node)
builder.add_node("error_handler", error_handler_node) # Add error handler node

# Set entry point
builder.set_entry_point("planner")

# Add edges
# Conditional routing from planner: check for error message immediately after planner
builder.add_conditional_edges(
    "planner",
    # Lambda function checks for 'error_message' key in state
    lambda state: "error_handler" if state.get("error_message") else "executor",
    {
        "executor": "executor",
        "error_handler": "error_handler"
    }
)


# Conditional routing from executor: loop back to executor or go to summarizer/error
# This router checks the state *after* the executor node completes its execution.
builder.add_conditional_edges(
    "executor",
    should_continue_execution, # This function determines next node
    {
        "executor": "executor",       # Loop back to executor if more tasks
        "summarizer": "summarizer",   # Go to summarizer if all tasks done
        "error_handler": "error_handler" # Go to error handler if error occurred (either from executor or earlier)
    }
)

# After summarizer or error, the graph ends
builder.add_edge("summarizer", END)
builder.add_edge("error_handler", END) # Error handler now goes to END

# Compile the graph
# Use checkpointer for state persistence, allowing you to resume runs.
# For simplicity, InMemorySaver is used. For production, use Redis, SQLite, etc.
workflow = builder.compile(checkpointer=InMemorySaver())

# Optional: Visualize the graph (requires graphviz and pydot)
# try:
#     from IPython.display import Image, display
#     display(Image(workflow.get_graph().draw_mermaid_png()))
# except Exception:
#     print("Could not display graph. Ensure graphviz and pydot are installed.")


# --- Hands-on Exercises Using LangGraph ---

print("--- LangGraph Multi-Agent Trip Planning Workflow ---")

# Use a checkpointer for state persistence, allowing you to resume runs.
# For simplicity, InMemorySaver is used. For production, use Redis, SQLite, etc.
# The workflow was already compiled with the checkpointer above, no need to re-compile here.

# --- Exercise 1: Multi-city trip with flights and hotels ---
user_request_1 = "Plan a trip for me: I need flights from Montreal to Paris on August 15, 2025, a hotel in Paris for August 15-20, 2025 for 2 guests, then flights from Paris to Rome on August 20, 2025, and finally a hotel in Rome from August 20-22, 2025 for 2 guests."
print(f"\nUser Request: {user_request_1}")
# `invoke` will run the entire graph
# FIX: Add config with a unique thread_id for the checkpointer
result_1 = workflow.invoke(
    {"request": user_request_1},
    config={"configurable": {"thread_id": "trip_planning_1"}} # Use a unique ID for this run
)
print("\n--- FINAL ITINERARY ---")
print(result_1['final_itinerary'])
print("-----------------------\n")

# --- Exercise 2: Single task request ---
user_request_2 = "Find flights from Montreal to Paris on August 15, 2025."
print(f"\nUser Request: {user_request_2}")
# FIX: Add config with a unique thread_id for the checkpointer
result_2 = workflow.invoke(
    {"request": user_request_2},
    config={"configurable": {"thread_id": "trip_planning_2"}} # Use another unique ID
)
print("\n--- FINAL ITINERARY ---")
print(result_2['final_itinerary'])
print("-----------------------\n")

# --- Exercise 3: Request that might trigger an error (e.g., unparseable date or location for a service) ---
# Modify a specialized agent to throw an error for certain inputs to test.
# For this demo, let's make an input that planner might struggle with or executor gets no data from.
# Or you can add a deliberate error in one of the service.run methods for testing.
# Let's simulate an executor error if "unknown_city" is in the request.
class FaultyHotelBookingAgent(HotelBookingAgent):
    def run(self, query: str) -> str:
        if "unknown_city" in query.lower():
            print("  [FaultyHotelBookingAgent] Simulating an API error for unknown_city.")
            # Return a string that indicates an error, matching the check in executor_node
            return "ERROR: API returned an error for unknown city data."
        # Call the original run method for other inputs
        return super().run(query)

# Temporarily replace the hotel_service with the faulty one for testing purposes
temp_hotel_service = hotel_service
hotel_service = FaultyHotelBookingAgent(llm) # Inject the faulty agent

user_request_3 = "I need a hotel in unknown_city from 2025-08-15 to 2025-08-20 for 1 guest."
print(f"\nUser Request: {user_request_3}")
# FIX: Add config with a unique thread_id for the checkpointer
result_3 = workflow.invoke(
    {"request": user_request_3},
    config={"configurable": {"thread_id": "trip_planning_3"}} # Use yet another unique ID
)
print("\n--- FINAL ITINERARY ---")
print(result_3['final_itinerary'])
print("-----------------------\n")

# Revert to original hotel service
hotel_service = temp_hotel_service

--- LangGraph Multi-Agent Trip Planning Workflow ---

User Request: Plan a trip for me: I need flights from Montreal to Paris on August 15, 2025, a hotel in Paris for August 15-20, 2025 for 2 guests, then flights from Paris to Rome on August 20, 2025, and finally a hotel in Rome from August 20-22, 2025 for 2 guests.

[Node: Planner] Breaking down the request...
[Node: Planner] Error planning tasks: Error planning tasks: 'Input to ChatPromptTemplate is missing variables {\'\\\\"location\\\\"\'}.  Expected: [\'\\\\"location\\\\"\', \'request\'] Received: [\'request\']\nNote: if you intended {\\"location\\"} to be part of the string and not a variable, please escape it with double curly braces like: \'{{\\"location\\"}}\'.\nFor troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_PROMPT_INPUT '

[Node: Error Handler] Processing error: Error planning tasks: 'Input to ChatPromptTemplate is missing variables {\'\\\\"location\\\\"\'}.  Expected: [\'\\\\"loc

Explanation and Key Learnings:

1. Graph State (TravelGraphState): This is the central piece of LangGraph. It defines the schema of the information that is passed between nodes. Each node receives the current state, updates it, and returns the modified state.

2. Nodes (Functions):
planner_node: Takes the initial request, uses an LLM to parse it into structured tasks, and updates state['tasks'] and state['current_task_index'].
executor_node: This is the workhorse. It reads the current_task_index from the state, executes the corresponding task by calling our specialized flight_service.run() or hotel_service.run() (simulating API calls to other agents), and then appends the results to state['flight_results'] or state['hotel_results']. It increments current_task_index.
summarizer_node: Takes all accumulated flight_results and hotel_results from the state and uses another LLM call to synthesize the final_itinerary.
error_handler_node: A dedicated node to provide a graceful response if an error occurred during processing.

3. Conditional Edges (should_continue_execution): This function is the "router" that defines the graph's flow.

* * It's called after the executor node.
* * It inspects the state (specifically current_task_index and error_message).
* * If error_message is present, it routes to error_handler.
* * If there are more tasks to execute (current_task_index < len(tasks)), it routes back to the executor node, creating a loop.
* * Otherwise (all tasks processed), it routes to the summarizer node.

4. Cycles (Looping Behavior): The edge executor -> should_continue_execution -> executor forms a cycle, enabling the graph to iteratively process each task identified by the planner.

5. Stateful Transitions: The state is continuously updated by each node. This allows subsequent nodes (and even subsequent iterations of the same node) to access the results and context from previous steps. For example, the summarizer_node receives all results accumulated by multiple runs of the executor_node.

6. Integrating Memory: The TravelGraphState itself serves as the active memory for the current workflow execution. For persistence across sessions, workflow.compile() accepts a checkpointer (e.g., InMemorySaver for local, or more robust options like SqliteSaver or Redis for production). This allows you to invoke with a thread_id and resume a previous conversation.

7. Node-based Task Design: Each node has a clear, singular responsibility (planning, executing one task, summarizing, handling errors), making the system modular and easier to debug.

LangGraph provides a powerful, explicit, and visual way to design complex agentic workflows, moving beyond simple linear chains to truly dynamic and robust multi-agent systems for challenging tasks like flight planning.